![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
### Script Para rodar em toda a base de São Paulo
# Importando nossas bibliotecas

In [1]:
# import our data and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

from tqdm import tqdm
import time

import spacy
from spacy import displacy
import pt_core_news_sm

import csv
import gzip
import io
from unicodedata import normalize

import math

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Definindo o diretório. Percorreremos em todo o diretório e vamos filtrar: 
<ul>
<li> 1) Número de Processo </li>
<li> 2) Assunto </li>
<li> 3) Autor </li>
<li> 4) Réu </li>
<li> 5) Classe </li>
<li> 6) Valores </li>
</ul>
<!-- Count the separator ' - ' in `Conteúdo` of sp_data -->

In [2]:
# Queremos iterar sobre o diretório e para cada dataset filtrar as informações relevantes

# diretório em que estão todos os documentos de São Paulo
DIR = '/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/data/CSV IC DIREITO TUDO SP'

In [3]:
nlp = pt_core_news_sm.load()

In [4]:
chunksize = 200000

j = 0

# função para salvar um arquivo json em outro arquivo ja existente
def to_json_append(df,file):
    '''
    df = dataframe que será adicionado ao final de um arquivo json ja criado
    file = nome do arquivo a ser adicionado o dataframe
    '''
    
    path = 'Documentos Filtrados/'
    df.to_json('tmp.json',orient='records',lines=True, force_ascii=False)
    #append
    f=open('tmp.json','r')
    k=f.read()
    f.close()
    f=open(path+file,'a+', encoding='utf-8')
    f.write('\n') #Prepare next data entry
    f.write(k)
    f.close()

def encode(name):
    ascii_name = normalize("NFKD", name).encode("ascii", errors="ignore").decode("ascii")
    return ascii_name.upper()

def load_data():
    fobj = io.TextIOWrapper(gzip.open("/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/Nomes pessoas/nomes.csv.gz"), encoding="utf-8")
    csv_reader = csv.DictReader(fobj)
    data = {
        row["first_name"]: row["classification"]
        for row in csv_reader
    }
    fobj.close()
    return data

name_data = load_data()

def classify_download(name):
    encoded_name = encode(name)
    try:
        return name_data[encoded_name]
    except:
        return 'Nao Nome'

def Search_Reus_Autores(conteudo_all, partes_all):
    Autores = []
    Reus = []
    for i,conteudo in enumerate(conteudo_all):
        texto_interesse = partes_all[i] + ' - ' + conteudo_all[i]
        texto = nlp(texto_interesse)

        primeiro_ponto_final = texto_interesse.find('.')
        primeira_virgula = texto_interesse.find(',')
        separador_oficial = texto_interesse.find(' X ')
        if (separador_oficial == -1):
            separador_oficial = texto_interesse.find(' x ')
        separadores = []

        for match in re.finditer(' - ',texto_interesse):

            if (primeiro_ponto_final != -1 and primeira_virgula != -1):

                if(match.start() < primeiro_ponto_final or match.start() < primeira_virgula):
                    separadores.append(match.start())

            else:
                separadores.append(match.start())

        autor = []
        reus = []
        if (separador_oficial > -1):

            primeiro_espaço = -1
            primeiro_espaço_depois = -1

            for match in re.finditer(' - ', texto_interesse):
                if (match.start() < separador_oficial):
                    primeiro_espaço = match.start()
                if (match.start() > separador_oficial and primeiro_espaço_depois == -1):
                    primeiro_espaço_depois = match.start()

            if (primeiro_espaço > -1):
                autor.append(texto_interesse[primeiro_espaço:separador_oficial])
            else:
                autor.append(texto_interesse[:separador_oficial])

            if (primeiro_espaço_depois > -1):
                reus.append(texto_interesse[separador_oficial + 3:primeiro_espaço_depois])
            else:

                if (primeira_virgula < primeiro_ponto_final):
                    reus.append(texto_interesse[separador_oficial + 3:primeira_virgula])
                else:
                    reus.append(texto_interesse[separador_oficial + 3:primeiro_ponto_final])

        else:
            for nome_proprio in texto.ents:

                start_char_autor = 0

                if len(autor) < 1:

                    if separadores != []:

                        autor.append(texto_interesse[:separadores[0]])
                    else:
                        autor.append(nome_proprio)

                else:
                    if (nome_proprio.start_char < primeiro_ponto_final and nome_proprio.start_char < primeira_virgula):

                        if len(separadores) > 1:
                            if (nome_proprio.start_char >= separadores[0] and nome_proprio.start_char <= separadores[1]):

                                reus.append(texto_interesse[separadores[0] + 3:separadores[1]])

                                del separadores[0]
                        else:
                            pass

        Autores.append(autor)
        Reus.append(reus)
    return Autores, Reus

In [ ]:
if __name__ == "__main__":
    for file in tqdm(os.listdir(DIR)):

        is_json = re.search(r'.*\.json', file,re.IGNORECASE) # vendo se o arquivo no diretório é do tipo json

        if(is_json):
            print('Executando: ' + file)

            # lendo o dataset em 200.000 registros por vez
            time_inicio_chunk = time.time()
            for df in pd.read_json(DIR+'/'+file, chunksize=chunksize, lines=True):

                # ---------------------------------------------------
                # Contando os separadores no conteúdo
                tempo_sepadores = time.time()
                count_separator = [] # this is a vector which contains the frequency of ' - ' in each Document

                count_Counteudo_length = [] # this is a vector which contains the lenght of "Conteúdo" in each Document

                # for each Document in data, count the frequency of ' - ' and the len of "Conteúdo"
                for i, Document in enumerate(df['Conteúdo']):
                    count_separator.append(Document.count(' - '))

                data = np.array(count_separator) # vector which contains the amount of ' - ' in each document
                tempo_sepadores = time.time() - tempo_sepadores       
                
                # ---------------------------------------------------
                # FIM

                # ---------------------------------------------------
                # Separando as possiveis classes, assuntos, etc ...
                tempo_Separar_tudo = time.time()
                Doc_series = pd.Series(df['Conteúdo'])

                ind_range = (data > 4)

                data_more_then_4 = df.loc[ind_range,('Conteúdo')]

                positions_all = []

                processos_all = []

                classes_all = []

                assuntos_all = []

                partes_all = []

                conteudo_all = []

                advs_all = []

                for document in data_more_then_4:

                    # ----------------------------------------------------------
                    # Filtering the processes
                    # ----------------------------------------------------------

                    position = re.search(r' - ', document)
                    processos_all.append(document[0:position.start()])

                    doc_no_processos = document[position.end():]

                    # ----------------------------------------------------------
                    # Filtering the classes
                    # ----------------------------------------------------------
                    position = re.search(r' - ', doc_no_processos)
                    classes_all.append(doc_no_processos[0:position.start()])

                    doc_no_classes_to = doc_no_processos[position.end():]

                    # ----------------------------------------------------------
                    # Filtering the assuntos
                    # ----------------------------------------------------------
                    position = re.search(r' - ', doc_no_classes_to)
                    assuntos_all.append(doc_no_classes_to[0:position.start()])

                    doc_no_assuntos_to = doc_no_classes_to[position.end():]

                    # ----------------------------------------------------------
                    # Filtering the parts
                    # ----------------------------------------------------------
                    position = re.search(r' - ', doc_no_assuntos_to)
                    partes_all.append(doc_no_assuntos_to[0:position.start()])

                    doc_no_parts_to = doc_no_assuntos_to[position.end():]

                    # ----------------------------------------------------------
                    # Filtering the content
                    # ----------------------------------------------------------

                    # ----------------------------------------------------------
                    # Filtering the ADV
                    # ----------------------------------------------------------
                    position = re.search(r"(?i)- ADV", doc_no_parts_to)
                    if position:
                        advs_all.append(doc_no_parts_to[position.start():])

                        conteudo_all.append(doc_no_parts_to[0: position.start()])
                    else:
                        conteudo_all.append(doc_no_parts_to)
                tempo_Separar_tudo = time.time() - tempo_Separar_tudo
                
                # ---------------------------------------------------
                # FIM

                # ---------------------------------------------------
                # Pegando o número certo do processo
                tempo_numero_processo = time.time()
                regex_processo = r"\d{6,7}.\d{2,3}.\d{4,5}.\d{1,2}.\d{2}\.\d{4,5}" # return the pattern 

                processos = df['Conteúdo'].str.findall(regex_processo)

                # take only the first time which that pattern was saw
                teste = processos.copy()
                teste = processos.str[0]

                # fazendo uma cópia do dataframe atual
                copy = df.copy()

                # todos os processos do dataframe atual são atualizados, porém
                df['Processo'] = teste

                # naquelas posições em que não houve êxito em achar o número do processo,
                # deixe como estava anteriormente
                df.loc[teste.isnull(),('Processo')] = copy.loc[teste.isnull(),('Processo')]
                tempo_numero_processo = time.time() - tempo_numero_processo
                
                # ---------------------------------------------------
                # FIM

                # ---------------------------------------------------
                # Verificando os Assuntos no conteúdo
                print('Verificando assuntos ...') 
                tempo_assuntos = time.time()
                
                assuntos = ["Periculosidade","Contribuição / Taxa Assistencial","Contribuição Confederativa","Eleição","Registro de Entidade Sindical","Salário In Natura","Assédio Moral","Assédio Sexual","Cesta Básica","Contribuição Sindical","Comissão","Complemento Temporário Variável de Ajuste ao Piso de Mercado","Culpa Recíproca","Descontos Salariais - Devolução","Despedida / Dispensa Imotivada","Justa Causa / Falta Grave","Diárias","Dirigente Sindical","Empregado Público","Portador de HIV","Contrato Suspenso","Estabilidade decorrente de Norma Coletiva","Gestante","Membro da Cipa","Multa de 40% do FGTS","Fruição / Gozo","Indenização / Terço Constitucional","Supressão de Horas Extras Habituais - Indenização","Multa do Artigo 467 da CLT","Multa do Artigo 477 da CLT","Multa Prevista em Norma Coletiva","Plano de Demissão Voluntária","PIS - Indenização","Piso Salarial da Categoria / Salário Mínimo Profissional","Prêmio","Contribuição de Previdência Privada - Resgate","Plano de Saúde","Rescisão Indireta","Reajuste Salarial","Gorjeta","Salário Vencido / Retido","Salário Família","Salário Paternidade","Salário Por Fora - Integração","Salário Por Safra","Seguro de Vida","Indenização","Liberação das Guias","Tarefa","Tíquete Alimentação","Vale Transporte","Contrato de Aprendizagem","Cooperativa de Trabalho","Contrato de Estágio","Acidente de Trabalho","Abono","Adicional de Transferência","Ajuda de Custo","Complementação de Aposentadoria / Pensão","AVISO PRÉVIO","Anistia","Estabilidade Acidentária","Abono Pecuniário","Décimo Terceiro Salário","Salário por Equiparação / Isonomia","Tomador de Serviços / Terceirização","Homicídio Simples","Homicídio Privilegiado","Homicídio Qualificado","Induzimento, Instigação ou Auxílio a Suicídio","Infanticídio","Leve","Seguida de Morte","Perigo de contágio de moléstia grave","Perigo para a vida ou saúde de outrem","Abandono de incapaz","Omissão de socorro","Rixa","Calúnia","Difamação","Injúria","Constrangimento ilegal","Ameaça","Seqüestro e cárcere privado","Redução a condição análoga à de escravo","Violação de domicílio","Violação de correspondência","Sonegação ou destruição de correspondência","Violação de comunicação telegráfica, radioelétrica ou telefônica","Violação de Correspondência comercial","Divulgação de segredo","Violação do segredo profissional","Furto","Furto Qualificado","Furto de coisa comum","Roubo","Extorsão","Extorsão mediante seqüestro","Extorsão indireta","Alteração de limites","Usurpação de águas","Esbulho possessório","Dano","Introdução ou abandono de animais em propriedade alheia","Dano em coisa de valor artístico, arqueológico ou histórico","Alteração de local especialmente protegido","Apropriação indébita Previdenciária","Estelionato","Estelionato Majorado","Outras fraudes","Fraude à execução","Receptação","Apropriação indébita","Apropriação de Coisa Havida por Erro, Caso Fortuito ou Força da Natureza","Violação de direito autoral","Atentado contra a liberdade de trabalho","Atentado contra a liberdade de contrato de trabalho e boicotagem violenta","Atentado contra a liberdade de associação","Paralisação de trabalho, seguida de violência ou perturbação da ordem","Paralisação de trabalho de interesse coletivo","Invasão de estabelecimento industrial, comercial ou agrícola. Sabotagem","Frustração de direitos assegurados por lei trabalhista","Exercício de atividade com infração de decisão administrativa","Aliciamento para fins de emigração","Aliciamento de trabalhadores de um local para outro do território nacional","Frustração de lei sobre a nacionalização do trabalho","Destruição / Subração / Ocultação de Cadáver","Impedimento ou Perturbação de Cerimônia Funerária","Vilipêndio a Cadáver","Violação a Sepultura","Ultraje / Impedimento ou Perturbação de Culto Religioso","Estupro","Atentado Violento ao Pudor","Atentado ao Pudor Mediante Fraude","Corrupção de Menores","Ultraje Público ao Pudor (Ato/Escrito Obsceno)","Mediação para Servir a Lascívia de Outrem","Abandono Material","Abandono Intelectual","Entrega de Filho Menor a Pessoa Inidônea","Bigamia","Conhecimento Prévio de Impedimento","Induzimento a erro essencial e ocultação de impedimento","Simulação de Casamento","Simulação de Autoridade para Celebração de Casamento","Parto Suposto","Registro de Nascimento Inexistente","Sonegação do Estado de Filiação","Induzimento a Fuga, Entrega Arbitrária ou Sonegação de Incapazes","Subtração de Incapazes","Incêndio","Explosão","Uso de gás tóxico ou asfixiante","Inundação / Perigo de Inundação","Desabamento ou desmoronamento","Subtração, ocultação ou inutilização de material de salvamento","Difusão de doença ou praga","Desastre ferroviário / Perigo de Desastre Ferroviário","Atentado contra a segurança de transporte público","Arremesso de projétil","Atentado contra a segurança de serviços de utilidade pública","Interrupção /perturbação de serviços telegráficos/telefônicos","Epidemia","Emprego de Processo Proibido/Substância não permitida","Invólucro ou recipiente com falsa indicação","Falsificação / Corrupção / Adulteração / Alteração de produto destinado a fins terapêuticos ou medicinais","Exercício Ilegal da Medicina, Arte Dentária ou Farmacêutica","Charlatanismo","Corrupção ou Poluição de Água Potável","Curandeirismo","Envenenamento de Água Potável / Substância Alimentícia ou Medicinal","Falsificação / Corrupção / Adulteração / Alteração de Substância ou Produtos Alimentícios","Infração de Medida Sanitária Preventiva","Omissão de Notificação de Doença","Outras Substâncias Nocivas a Saúde Pública","Substância Destinada à Falsificação","Quadrilha ou Bando","Moeda Falsa / Assimilados","Petrechos para falsificação de moeda","Emissão de título ao portador sem permissão legal","Falsificação de papéis públicos","Falsificação do selo ou sinal público","Falsificação de documento público","Falsificação de documento particular","Falsidade ideológica","Falso reconhecimento de firma ou letra","Certidão ou atestado ideologicamente falso","Falsidade material de atestado ou certidão","Falsidade de atestado médico","Reprodução ou adulteração de selo ou peça filatélica","Uso de documento falso","Supressão de documento","Falsificação do sinal empregado no contraste de metal precioso ou na fiscalização alfandegária, ou para outros fins","Falsa identidade","Fraude de lei sobre estrangeiros","Petrechos de Falsificação de Papéis Públicos","Adulteração de Sinal Identificador de Veículo Automotor","Peculato","Peculato mediante erro de outrem","Extravio, sonegação ou inutilização de livro ou documento","Emprego irregular de verbas ou rendas públicas","Concussão","Excesso de exação","Corrupção passiva","Facilitação de contrabando ou descaminho","Prevaricação","Condescendência criminosa","Advocacia administrativa","Violência arbitrária","Abandono de função","Exercício funcional ilegalmente antecipado ou prolongado","Violação do sigilo funcional","Violação do sigilo de proposta de concorrência","Usurpação de função pública","Resistência","Tráfico de influência","Corrupção ativa","Impedimento, perturbação ou fraude de concorrência","Inutilização de edital ou de sinal","Subtração ou inutilização de livro ou documento","Desobediência","Desacato","Contrabando ou descaminho","Reingresso de estrangeiro expulso","Denunciação caluniosa","Comunicação falsa de crime ou de contravenção","Auto-acusação falsa","Falso testemunho ou falsa perícia","Coação no curso do processo","Exercício arbitrário das próprias razões","Fraude processual","Favorecimento pessoal","Favorecimento real","Exercício arbitrário ou abuso de poder","Fuga de pessoa presa ou submetida a medida de segurança","Evasão mediante violência contra a pessoa","Arrebatamento de preso","Motim de presos","Patrocínio infiel","Patrocínio simultâneo ou tergiversação","Sonegação de papel ou objeto de valor probatório","Exploração de prestígio","Violência ou fraude em arrematação judicial","Desobediência a decisão judicial sobre perda ou suspensão de direitos","Inserção de dados falsos em sistema de informações","Modificação ou alteração não autorizada de sistema de informações","Sonegação de contribuição previdenciária","Crimes de Responsabilidade","Crimes contra a Economia Popular","Crimes de Abuso de Autoridade","Tráfico de Drogas e Condutas Afins","Responsabilidade Criminal por Danos Nucleares","Crimes contra o Serviço Postal e o Serviço de Telegrama","Crimes contra a Segurança Nacional, a Ordem Política e Social","Crimes contra o Sistema Financeiro Nacional","Crimes Resultante de Preconceito de Raça ou de Cor","Crimes contra a Ordem Tributária","Crimes contra a Ordem Econômica","Crimes contra as Relações de Consumo","Representação caluniosa","Crimes contra a Fauna","Crimes contra a Flora","Da Poluição","Agrotóxicos","Atividades Nucleares","Caça","Liberação ou Descarte de OGM (Organismo Geneticamente Modificado)","Pesca","ou Ocultação de Bens, Direitos ou Valores","Crimes contra as Telecomunicações","Crimes Previstos na Lei de Estrangeiros","Crimes de Tortura","Crimes de Trânsito","Crimes do Sistema Nacional de Armas","Crimes de Imprensa","Crimes Previstos no Estatuto da criança e do adolescente","Crimes contra o mercado de capitais","Crimes de genocídio","Sigilo Telefônico","Crimes da Lei de licitações","Crimes da Lei de remoção de órgãos e tecidos","Crimes praticados contra os índios e a cultura indígena","Recusa, retardamento ou omissão de dados técnicos indispensáveis à propositura de Ação Civil Pública","Crimes Agrários","Crimes relacionados à Comissão Parlamentar de Inquérito","Crime de Quebra de Sigilo Financeiro","Crimes do Código Brasileiro de Telecomunicações","Crimes Previstos no Estatuto do Idoso","Parcelamento do solo urbano","Crimes Falimentares","Trancamento","Fiança","Prisão Preventiva","Recebimento","Rejeição","Aplicabilidade","Revisão","Anulação","Restituição / Indenização de Despesa","Representação Sindical","Direito Autoral","Patente","Desenho Industrial","Marca","Defeito, nulidade ou anulação","Evicção ou Vicio Redibitório","Cessão de Crédito","Câmbio","Parceria Agrícola e/ou pecuária","Representação comercial","Atraso de vôo","Cancelamento de vôo","Overbooking","Extravio de bagagem","Equivalência salarial","Quitação","Reajuste de Prestações","Transferência de Financiamento (contrato de gaveta)","Sustação/Alteração de Leilão","Seguro","Revisão do Saldo Devedor","Assembléia","Eleição","Extinção","Assembléia","Eleição","Extinção","Apuração de haveres","Constituição","Dissolução","Desconsideração da Personalidade Jurídica","Ingresso e Exclusão dos Sócios na Sociedade","Responsabilidade dos sócios e administradores","Transferência de cotas","Anulação","Requisitos","Cédula de Crédito Bancário","Cédula de Crédito à Exportação","Cédula de Crédito Comercial","Cédula de Crédito Industrial","Cédula de Crédito Rural","Cédula de Produto Rural","Cédula Hipotecária","Cheque","Debêntures","Duplicata","Letra de Câmbio","Nota de Crédito Comercial","Nota de Crédito Industrial","Nota de Crédito Rural","Nota Promissória","Warrant","Recuperação extrajudicial","Autofalência","Concurso de Credores","Liquidação","Revogação de atos praticados em prejuízo de credores e da massa","Bolsa de Valores","Engajamento e Profissionais Marítimos","Seguros Marítimos","Aluguel de Embarcações (Fretamento E Carta Partida)","Mínimo","Advogados","Aeronautas","Artistas","Atleta Profissional","Bancários","Cabineiros de Elevador","Doméstico","Engenheiro, Arquiteto e Engenheiro Agrônomo","Jornalistas","Marítimos","Digitadores / Mecanógrafos / Datilógrafos","Médicos","Mineiros de Subsolos","Músicos Profissionais","Operadores de Carga e Descarga (Estiva e Capatazia)","Operadores Cinematográficos","Professores","Químicos","Radialistas","Serviços de Telefonia ou Telegrafia","Trabalhadores em Petróleo","Vigia e Vigilantes","Grupo Econômico","Crime Tentado","Grave","Gravíssima","Privilegiada","Furto Privilegiado","Furto de Veículo Automotor a ser Transportado para outro Estado ou Exterior","Roubo Majorado","Latrocínio","Extorsão mediante Seqüestro Seguida de Morte","Extorsão mediante Sequestro Seguida de Lesão Corporal Grave","Supressão ou Alteração de Marca em Animais","Dano Qualificado","Apropriação de Tesouro","Apropriação de Coisa Achada","Abandono","Acidentes da Navegação","Registro / Cadastro do Armador","Arresto de Embarcação","Assistência / Salvamento","Avaria","Responsabilidade do Comandante ou Capitão","Inscrição / Registro da Embarcação","Hipoteca Marítima","Créditos / Privilégios Marítimos","Agenciamento","Corretagem de Embarcação","Praticagem","Prescrição e Decadência","Edição","Exoneração","Revisão","Busca e Apreensão de Menores","Guarda","Investigação de Paternidade","Regulamentação de Visitas","Nulidade / Anulação","Nulidade e Anulação de Testamento","Nulidade e Anulação de Partilha e Adjudicação de Herança","Sub-rogação de Vinculo","Exclusão de herdeiro ou legatário","Petição de Herança","Deserdação","Disposição de coisa alheia como própria","Alienação ou Oneração Fraudulenta de Coisa Própria","Defraudação de Penhor","Fraude na Entrega de Coisa","Fraude para Recebimento de Indenização ou Valor de Seguro","Fraude no Pagamento por Meio de Cheque","Duplicata Simulada","Abuso de Incapazes","Induzimento à Especulação","Fraude no Comércio","Fraudes e Abusos na Fundação ou Administração de Sociedade por Ações","Receptação Qualificada","Posse Sexual Mediante Fraude","Assédio Sexual","Favorecimento da Prostituição","Casa de Prostituição","Rufianismo","Tráfico Internacional de Pessoas","Tráfico Interno de Pessoas","Fabrico / Fornecimento / Aquisição / Posse ou Transporte de Explosivos ou Gás Tóxico ou Asfixiante","Crime Culposo","Venda de Produtos ou Substância nas Condições dos Artigos 274 e 275","Medicamento em Desacordo com Receita Médica","Incitação ao Crime","Apologia de Crime ou Criminoso","Corrupção Ativa em Transação Comercial Internacional","Tráfico de Influência em Transação Comercial Internacional","Crimes contra Patente de Invenção","Crimes contra os Desenhos Industriais","Crimes contra as Marcas","Crimes Cometidos por Meio de Marca, Título de Estabelecimento e Sinal de Propaganda","Crimes contra Indicações Geeográficas e Demais Indicações","Crimes de Concorrência Desleal","Crime Contra a Administração da Justiça - Lei da Ação de Alimentos","Fabricação de Objeto Destinado a Produção de Drogas e Condutas Afins","Indução, Instigação ou Auxílio ao Uso de Drogas","Oferecimento de Drogas para Consumo Conjunto","Associação para a Produção e Tráfico e Condutas Afins","Financiamento ou Custeio de Produção ou Tráfico de Drogas","Colaboração com Grupo, Organização ou Associação Destinados à Produção ou Tráfico de Drogas","Prescrição Culposa de Drogas","Condução de Embarcação ou Aeronave sob Efeito de Drogas","Contratação de Operação de Crédito","Inscrição de Despesas Não Empenhadas","Assunção de Obrigação no Último Ano do Mandato ou Legislatura","Ordenação de Despesa Não Autorizada","Prestação de Garantia Graciosa","Não Cancelamento de Restos a Pagar","Aumento de Despesa com Pessoal no Último Ano do Mandato ou Legislatura","Oferta Pública ou Colocação de Títulos no Mercado","Perigo de Contágio Venéreo","Isenção","Incidência sobre PDV","Incidência sobre Aposentadoria","Incidência sobre Aplicações Financeiras","Incidência sobre Licença-Prêmio/Abono/Indenização","Retido na fonte","Incidência sobre Férias Compensadas","Incidência sobre Proventos de Previdência Privada","Incidência sobre Participação nos Lucros","Incidência sobre Lucro Imobiliário","Incidência sobre 1/3 de férias (art. 7º, XVII da CF)","Incidência sobre 13° Salário","Incidência sobre Função Comissionada","Incidência sobre Hora Extra","Incidência decorrente de Desligamento de Plano de Previdência Privada","Incidência decorrente de Liquidação de Entidade de Previdência Privada","Microempresa","Demonstrações Financeiras (DCTF)","Incidência sobre Aplicações Financeiras","Retido na fonte","Incidência sobre Lucro","Cooperativa","Compensação de Prejuízos","Importação de bens usados","Drawback","IE/ Imposto sobre Exportação","ICMS/Importação","Incidência sobre Ações de Companhias Abertas","ITR/ Imposto Territorial Rural","ISS/ Imposto sobre Serviços","IPTU/ Imposto Predial e Territorial Urbano","IPVA - Imposto Sobre Propriedade de Veículos Automotores","ITBI - Imposto de Transmissão Intervivos de Bens Móveis e Imóveis","ITCD - Imposto de Transmissão Causa Mortis","Taxa Anual por Hectare","Taxa de Armazenamento","Taxa de Despacho Aduaneiro","Taxa de Exploração Mineral","Taxa de Fiscalização Ambiental","Taxa de Guia de Importação","Taxa de Licença de Importação","Taxa de Melhoramento de Portos","Taxa de Permanência","Taxa de Saúde Suplementar","Taxa de Fiscalização da Comissão de Valores Mobiliários","Taxa Judiciária","Taxa de Aferição de Equipamentos de Metrologia","Estaduais","Aquisição de veículos automotores","Aquisição de combustíveis","Aquisição de passagens e moedas - Viagem Exterior","Energia Elétrica","Responsabilidade Tributária do Sócio-Gerente / Diretor / Representante","Substituição Tributária","DIMOB/Declaração de Informações sobre Atividades Imobilíárias","CNPJ/Cadastro Nacional de Pessoas Jurídicas","CPF/Cadastro de Pessoas Físicas","Obrigação Acessória","Carta de fiança","Parcelamento","Compensação com Apólices da Dívida Pública","Compensação","TDA/Títulos da Dívida Agrária","Anistia","Denúncia espontânea","Lançamento","Expedição de CND","Expedição de Certidão Positiva de Débito com Efeito de Negativa","Certificado de Regularidade - FGTS","Incentivos fiscais","Anulação de Débito Fiscal","Juros/Correção Monetária","Prazo de Recolhimento","Repetição de indébito","Crédito Prêmio","Crédito Presumido","Fato Gerador/Incidência","Alíquota Zero","Alíquota Progressiva","Índice da Alíquota","Creditamento","Depósito Prévio ao Recurso Administrativo","Arrolamento de Bens","Utilização de Dados Relativos à CPMF para Fins de Fiscalização","Liberação de mercadorias","Levantamento de depósito","Cadastro de Inadimplentes - CADIN","Autorização para Impressão de Documentos Fiscais - AIDF","Liberação de Veículo Apreendido","Perdimento de Bens","Contribuições de Melhoria","Finsocial","Salário-Educação","Seguro Acidentes do Trabalho","Funrural","Contribuição INCRA","PASEP","Construção Civil","Contribuições para o SEBRAE, SESC, SENAC, SENAI e outros","Conselhos Regionais e Afins (Anuidade)","Contribuição Sindical","Servidores Ativos","Servidores Inativos","Mandato Eletivo/Lei 9.506/97","Aposentadoria/Retorno aoTrabalho","Cargo em Comissão","Tempo de Serviço Urbano/Contribuições não Recolhidas","Tempo de Serviço Rural/Contribuições não Recolhidas","Gratificação Natalina/13º Salário","Produção Agropecuária","Indenização Trabalhista","Reembolso auxílio-creche","Contribuição sobre a folha de salários","Devolução de contribuições previdenciárias pagas além do teto","1/3 de férias","Auxílio-Alimentação","Custeio de Assistência Médica","Hora Extra","Massa Falida- Recolhimento","Nota Fiscal ou Fatura","Salário-Maternidade","Contribuição sobre Nota Fiscal de Execução de Serviços","Contribuição de Autônomos, Empresários (Pró-labore) e Facultativos","FNT/Fundo Nacional de Telecomunicações","ATP/Adicional de Tarifa Portuária","AITP/Adicional de Indenização do Trabalhador Portuário Avulso","Adicional de Tarifa Aeroportuária","AFRMM/Adicional ao Frete para Renovação da Marinha Mercante","FUNDEF/Fundo de Manutenção e Desenvolvimento do Ensino Fundamental e de Valorização do Magistério","Seguro Apagão (Lei 10.438/02)","Contribuição sobre licença de uso ou transferência de tecnologia","Contribuição de Intervenção no Domínio Econômico (CIDE)","FUNDAF/Fundo Especial de Desenvolvimento e Aperfeiçoamento das Atividades de Fiscalização","FUSEX/FUNSA/FUSMA/Fundo de Saúde das Forças Armadas","Contribuição de Iluminação Pública","IUEE/Imposto Único sobre Energia Elétrica","FGTS/Fundo de Garantia Por Tempo de Serviço","Contribuição sobre Açúcar e Álcool","CPMF/Contribuição Provisória sobre Movimentação Financeira","Discriminação Tributária MERCOSUL","REFIS/Programa de Recuperação Fiscal","PAES/Parcelamento Especial","SIMPLES","Super SIMPLES","Aposentadoria por Invalidez","Urbana (Art. 48/51)","Rural (Art. 48/51)","Aposentadoria por Tempo de Serviço (Art. 52/4)","Aposentadoria Especial (Art. 57/8)","Auxílio-Doença Previdenciário","Salário-Família (Art. 65/70)","Salário-Maternidade (Art. 71/73)","Pensão por Morte (Art. 74/9)","Auxílio-Reclusão (Art. 80)","Pecúlios (Art. 81/5)","Incapacidade Laborativa Parcial","Incapacidade Laborativa Permanente","Incapacidade Laborativa Temporária","Movimentos Repetitivos/Tenossinovite/LER/DORT","Redução da Capacidade Auditiva","Abono de Permanência em Serviço (Art. 87)","Ferroviário","Ex-combatentes","Renda Mensal Vitalícia","Aposentadoria por Tempo de Contribuição (Art. 55/6)","RMI pelo art. 202 CF/88 (média dos 36 últimos salários-de-contribuição)","RMI pelo art. 1º da Lei 6.423/77 - índices de atualização dos 24 1ºs salários-de-contribuição, anteriores aos 12 últimos","RMI pela equivalência entre Salário-de-Benefício e Salário-de-Contribuição","RMI sem incidência de Teto Limitador","RMI da pensão de dependente de ex-combatente","Benefício mínimo a partir da CF/88 (art. 201, § 2º CF/88)","Limitação do salário-de-benefício e da renda mensal inicial","RMI cuja salário-de-benefício supera menor valor teto","Escala de Salário-Base","Contribuição sobre vinte salários mínimos","Cálculo do Benefício de acordo com a Sistemática anterior à Lei 9.876/99","IRSM de Fevereiro de 1994(39,67%)","Alteração do coeficiente de cálculo de pensão","Alteração do coeficiente de cálculo do benefício","Cálculo do benefício de segurado especial de acordo com a Lei 9.876/99","Cálculo do fator previdenciário - Lei 9.876/99","Reajuste pela Súmula 260 do TFR","Art. 58 ADCT da CF/88","Utilização do PNS no Reajuste de Benefícios","Manutenção do Benefício pela equivalência salarial","Expurgos inflacionários sobre os benefícios","Salário Mínimo de Ncz$ 120,00 para junho/89","Reajuste de 147%","Reajustamento pelo IGP-DI","Aplicação de coeficiente de cálculo diverso do fixado na Lei n.º 8.213/91","Reajuste aplicado ao salário mínimo em setembro/94","Reajustamento pelo INPC","Art. 144 da Lei 8.213/91 e/ou diferenças decorrentes","Gratificação Natalina a partir da CF/88 (Art. 201, § 6º CF/88)","Abono da Lei 8.178/91","Índice de 4,02% da Lei 8.222/91","Desconto do DL 1.910/81","Descontos dos benefícios","Correção Monetária pela Súmula 71 TFR","Correção Monetária de Benefício pago com atraso","Reajuste conforme Portaria MPAS 714/1993","Habilitação e Reabilitação Profissional","Decadência/Prescrição","Declaração de Ausência","Atividade concomitante","Renúncia ao benefício","Demonstrativo das importâncias pagas","Períodos de Carência","Limite Mínimo de Idade","Perda da qualidade de segurado","Inclusão de Dependente","Data de Início de Benefício (DIB)","Cumulação","Parcelas de benefício não pagas","Concessão","Restabelecimento","Conversão","Averbação/Cômputo/Conversão de tempo de serviço especial","Averbação/Cômputo de tempo de serviço de segurado especial (regime de economia familiar)","Averbação/Cômputo de tempo de serviço  rural (empregado/empregador)","Averbação/Cômputo do tempo de serviço como aluno aprendiz","Averbação/Cômputo do tempo de serviço militar","Averbação/Cômputo de tempo de serviço de empregado doméstico","Averbação/Cômputo de tempo de serviço urbano","Certidão de Tempo de Serviço","Contagem Recíproca de Tempo de Serviço","Aquisição","Perda","Concessão de Naturalização","Cancelamento de Naturalização","Admissão / Entrada / Permanência / Saída","Asilo","Livre Trânsito MERCOSUL","Trabalhador Migrante ou Fronteiriço do MERCOSUL","Proteção Internacional a Direitos Humanos","Auxílio Direto","Mercosul","Tratado Internacional","Normas do Mercosul","Casamento e Divórcio","Sucessão de Bens de Estrangeiro","Prestação de Alimentos","Funcionamento de Empresa Estrangeira","Pessoa Jurídica Estrangeira","Contratos Internacionais","Inclusão Indevida em Cadastro de Inadimplentes","Oferta","Fixação","Turismo","Consórcio","Estabelecimentos de Ensino","Seguro","Assinatura Básica Mensal","Pulsos Excedentes","Empreitada","Representante Comercial Autônomo","Honorários Profissionais","Trabalhador Eventual","Trabalhador Avulso","Interdito Proibitório","Aeroviários","Rural","Aplicabilidade","Tutela e Curatela","Regime de Bens Entre os Cônjuges","Usufruto e Administração dos Bens de Filhos Menores","Bem de Família","Dissolução","Investigação de Maternidade","Adoção de Maior","União Homoafetiva","Administração de herança","Reconhecimento / Dissolução","Inventário e Partilha","Assunção de Dívida","Preferências e Privilégios Creditórios","Correção Monetária","Perdas e Danos","Cláusula Penal","Arras ou Sinal","Pagamento","Pagamento em Consignação","Pagamento com Sub-rogação","Imputação do Pagamento","Dação em Pagamento","Novação","Compensação","Confusão","Remissão das Dívidas","Promessa de Recompensa","Gestão de Negócios","Pagamento Indevido","Enriquecimento sem Causa","Registro de nascimento após prazo legal","Retificação de Nome","Cancelamento de Protesto","Acidente Aéreo","Auxílio-Doença Acidentário","Fornecimento de Energia Elétrica","Fornecimento de Água","Substituição do Produto","Rescisão do contrato e devolução do dinheiro","Abatimento proporcional do preço","Interpretação / Revisão de Contrato","Cartão de Crédito","Financiamento de Produto","Serviços Profissionais","Serviços Hospitalares","Indenização por Dano Material","Protesto Indevido de Título","Prestação Pecuniária","Perda de Bens e Valores","Prestação de Serviços à Comunidade","Interdição Temporária de Direitos","Limitação de Fim de Semana","Pena de Multa","Tratamento Ambulatorial","Internação","Quanto à Embarcação","Quanto à Carga","Clandestinos","Retificação de Data de Nascimento","Retificação de Sexo","Inscrição na Matrícula de Registro Torrens","Instituição de Bem de Família","Bloqueio de Matrícula","Por Remição","Por Terceiro Prejudicado","Registro de Óbito após prazo legal","Registro de Nascimento de Filho de Brasileiro Nascido no Exterior","Liberdade Provisória","Prisão em flagrante","Perempção","Inclusão/exclusão de Jurado","Deserção","Denegação","Falsidade","Direitos da Personalidade","Organizações Religiosas","Sucessão de Empregadores","Subempreitada","Sócio / Acionista","Acidente de Trabalho","Salário por Acúmulo de Cargo / Função","Salário Maternidade","Licenças e Folgas - Conversão em Pecúnia","Salário Substituição","Gratificação por Tempo de Serviço","Gratificação de Função","Décimo Terceiro Salário Proporcional","Férias Proporcionais","Indenização Adicional","Saldo de Salário","Indenização do artigo 478 da CLT","Imunidade de Jurisdição","Assistência Judiciária Gratuita","Intervenção de Terceiros","Litigância de Má-Fé","Litisconsórcio","Sucessão Processual","Procuração","Representação em Juízo","Ministério Público","Dos Auxiliares da Justiça","Prazo","Valor da Causa","Suspensão do Processo","Modificação ou Alteração do Pedido","Antecipação de Tutela / Tutela Específica","Erro de Procedimento","Revelia","Vícios Formais da Sentença","Efeitos","Preparo / Deserção","Desconfiguração de Justa Causa","Cabimento","Levantamento de Valor","Penhora / Depósito/ Avaliação","Multa de 10%","Inexequibilidade do Título / Inexigibilidade da Obrigação","Expropriação de Bens","Remição","Liminar","Honorários Periciais","Extinção da Execução","Concurso de Credores","Execução Previdenciária","Fraude à Execução","Imunidade de Execução","Sucessão","Capacidade Processual","Substituição da Parte","Causas Supervenientes à Sentença","Efeito Suspensivo / Impugnação / Embargos à Execução","Benefício de Ordem","Ato Atentatório à Dignidade da Justiça","Indenização do Prejuízo","Caução","Coligação","Transformação","Incorporação","Fusão","Cisão","Liquidação","Alteração de capital","Capacidade","Curadoria dos bens do ausente","Sucessão Provisória","Inclusão de associado","Exclusão de associado","Fiscalização","Exclusão de filiado","Fusão","Eleições - Diretórios","Ineficácia de atos  em relação à massa","Convolação de recuperação judicial em falência","Administração judicial","Classificação de créditos","Laudo Arbitral Internacional","Sustação de Protesto","Agêncie e Distribuição","Alienação Fiduciária","Arrendamento Rural","Arrendamento Mercantil","Cartão de Crédito","Comissão","Compra e Venda","Corretagem","Depósito","Doação","Empreitada","Fiança","Mandato","Troca ou Permuta","Prestação de Serviços","Seguro","Transação","Transporte de Coisas","Transporte de Pessoas","Estimatório","Comodato","Mútuo","Constituição de Renda","Jogo e Aposta","Compromisso","Contratos Bancários","Franquia","Locação de Móvel","Despejo para Uso Próprio","Despejo para Uso de Ascendentes e Descendentes","Despejo por Denúncia Vazia","Benfeitorias","Direito de Preferência","Conta de Participação","Simples","Nome Coletivo","Comandita Simples","Limitada","Anônima","Comandita por Ações","Cooperativa","Coligadas","Dependente de Autorização","Estrangeira","Homicídio Simples","Homicídio Privilegiado","Homicídio Qualificado","Induzimento, Instigação ou Auxílio a Suicídio","Infanticídio","Leve","Seguida de Morte","Grave","Gravíssima","Privilegiada","Perigo de contágio de moléstia grave","Perigo para a vida ou saúde de outrem (art. 132)","Abandono de incapaz (art. 133)","Omissão de socorro (art. 135)","Rixa (art. 137)","Perigo de Contágio Venéreo","Calúnia","Difamação","Injúria","Constrangimento ilegal (art. 146)","Ameaça (art. 147)","Seqüestro e cárcere privado (art. 148)","Redução a condição análoga à de escravo (art. 149)","Violação de domicílio (art. 150)","Violação de correspondência (art. 151, caput)","Sonegação ou destruição de correspondência (art. 151, § 1º, I)","Violação de comunicação telegráfica, radioelétrica ou telefônica (art. 151, § 1º, II, III e IV)","Violação de Correspondência comercial (art. 152)","Divulgação de segredo (art. 153)","Violação do segredo profissional (art. 154)","Furto (art. 155)","Furto Qualificado (Art. 155, § 4o.)","Furto de coisa comum (art. 156)","Roubo (art. 157)","Extorsão (art. 158)","Extorsão mediante seqüestro (art. 159)","Extorsão indireta (art. 160)","Alteração de limites (art. 161, caput)","Usurpação de águas (art. 161, § 1º, I)","Esbulho possessório (art. 161, § 1º, II)","Dano (art. 163)","Introdução ou abandono de animais em propriedade alheia (art. 164)","Dano em coisa de valor artístico, arqueológico ou histórico (art. 165)","Alteração de local especialmente protegido (art. 166)","Apropriação indébita Previdenciária (art. 168-A e Lei 8.212/91)","Estelionato","Estelionato Majorado (art. 171, § 3º)","Outras fraudes","Fraude à execução (art. 179)","Receptação","Apropriação indébita (art. 168, caput)","Apropriação de Coisa Havida por Erro, Caso Fortuito ou Força da Natureza (art.169)","Furto Privilegiado","Furto de Veículo Automotor a ser Transportado para outro Estado ou Exterior","Roubo Majorado","Latrocínio","Extorsão mediante Seqüestro Seguida de Morte","Extorsão mediante Sequestro Seguida de Lesão Corporal Grave","Supressão ou Alteração de Marca em Animais","Dano Qualificado","Apropriação de Tesouro","Apropriação de Coisa Achada","Disposição de coisa alheia como própria","Alienação ou Oneração Fraudulenta de Coisa Própria","Defraudação de Penhor","Fraude na Entrega de Coisa","Fraude para Recebimento de Indenização ou Valor de Seguro","Fraude no Pagamento por Meio de Cheque","Duplicata Simulada","Abuso de Incapazes","Induzimento à Especulação","Fraude no Comércio","Fraudes e Abusos na Fundação ou Administração de Sociedade por Ações","Receptação Qualificada","Violação de direito autoral","Atentado contra a liberdade de trabalho (art. 197)","Atentado contra a liberdade de contrato de trabalho e boicotagem violenta (art. 198)","Atentado contra a liberdade de associação (art. 199)","Paralisação de trabalho, seguida de violência ou perturbação da ordem (art. 200)","Paralisação de trabalho de interesse coletivo (art. 201)","Invasão de estabelecimento industrial, comercial ou agrícola. Sabotagem (art. 202)","Frustração de direitos assegurados por lei trabalhista (art. 203)","Exercício de atividade com infração de decisão administrativa (art. 205)","Aliciamento para fins de emigração (art. 206)","Aliciamento de trabalhadores de um local para outro do território nacional (art. 207)","Frustração de lei sobre a nacionalização do trabalho (art. 204)","Destruição / Subração / Ocultação de Cadáver","Impedimento ou Perturbação de Cerimônia Funerária (Art. 209)","Vilipêndio a Cadáver","Violação a Sepultura (Art. 210)","Ultraje / Impedimento ou Perturbação de Culto Religioso","Tráfico Internacional de Pessoa para Fim de Exploração Sexual","Estupro","Atentado Violento ao Pudor","Atentado ao Pudor Mediante Fraude","Corrupção de Menores","Ultraje Público ao Pudor (Ato/Escrito Obsceno)","Mediação para Servir a Lascívia de Outrem","Posse Sexual Mediante Fraude","Assédio Sexual","Favorecimento da Prostituição","Casa de Prostituição","Rufianismo","Tráfico Internacional de Pessoas","Tráfico Interno de Pessoas","Abandono Material","Abandono Intelectual","Entrega de Filho Menor a Pessoa Inidônea","Bigamia","Conhecimento Prévio de Impedimento","Induzimento a erro essencial e ocultação de impedimento","Simulação de Casamento","Simulação de Autoridade para Celebração de Casamento","Parto Suposto (Art. 242)","Registro de Nascimento Inexistente (Art. 241)","Sonegação do Estado de Filiação (Art. 243)","Induzimento a Fuga, Entrega Arbitrária ou Sonegação de Incapazes (Art. 248)","Subtração de Incapazes (Art. 249)","Incêndio","Explosão","Uso de gás tóxico ou asfixiante","Inundação / Perigo de Inundação","Desabamento ou desmoronamento (art. 256)","Subtração, ocultação ou inutilização de material de salvamento (art. 257)","Difusão de doença ou praga (art. 259)","Desastre ferroviário / Perigo de Desastre Ferroviário","Atentado contra a segurança de transporte público","Arremesso de projétil","Atentado contra a segurança de serviços de utilidade pública (art. 265)","Interrupção /perturbação de serviços telegráficos/telefônicos (art. 266)","Epidemia (art. 267)","Emprego de Processo Proibido/Substância não permitida","Invólucro ou recipiente com falsa indicação (art. 275)","Falsificação / Corrupção / Adulteração / Alteração de produto destinado a fins terapêuticos ou medicinais","Exercício Ilegal da Medicina, Arte Dentária ou Farmacêutica (art. 282)","Charlatanismo (Art. 283)","Corrupção ou Poluição de Água Potável (Art. 271)","Curandeirismo (Art. 284)","Envenenamento de Água Potável / Substância Alimentícia ou Medicinal (Art. 270)","Falsificação / Corrupção / Adulteração / Alteração de Substância ou Produtos Alimentícios","Infração de Medida Sanitária Preventiva (Art. 268)","Omissão de Notificação de Doença (Art. 269)","Outras Substâncias Nocivas a Saúde Pública (Art. 278)","Substância Destinada à Falsificação (Art. 277)","Fabrico / Fornecimento / Aquisição / Posse ou Transporte de Explosivos ou Gás Tóxico ou Asfixiante","Venda de Produtos ou Substância nas Condições dos Artigos 274 e 275","Medicamento em Desacordo com Receita Médica","Quadrilha ou Bando (art. 288)","Incitação ao Crime","Apologia de Crime ou Criminoso","Moeda Falsa / Assimilados","Petrechos para falsificação de moeda (art. 291)","Emissão de título ao portador sem permissão legal (art. 292)","Falsificação de papéis públicos (art. 293)","Falsificação do selo ou sinal público (art. 296)","Falsificação de documento público","Falsificação de documento particular (art. 298)","Falsidade ideológica (art. 299)","Falso reconhecimento de firma ou letra (art. 300)","Certidão ou atestado ideologicamente falso (art. 301, caput)","Falsidade material de atestado ou certidão (art. 301, § 1º)","Falsidade de atestado médico (art. 302)","Reprodução ou adulteração de selo ou peça filatélica (art. 303)","Uso de documento falso (art. 304)","Supressão de documento (art. 305)","Falsificação do sinal empregado no contraste de metal precioso ou na fiscalização alfandegária, ou para outros fins (art. 306)","Falsa identidade","Fraude de lei sobre estrangeiros","Petrechos de Falsificação de Papéis Públicos","Adulteração de Sinal Identificador de Veículo Automotor","Peculato (art. 312, caput e § 1º)","Peculato mediante erro de outrem (art. 313)","Extravio, sonegação ou inutilização de livro ou documento (art. 314)","Emprego irregular de verbas ou rendas públicas (art. 315)","Concussão (art. 316, caput)","Excesso de exação (art. 316, §§ 1º e 2º)","Corrupção passiva (art. 317)","Facilitação de contrabando ou descaminho (art. 318)","Prevaricação","Condescendência criminosa (art. 320)","Advocacia administrativa (art. 321)","Violência arbitrária (art. 322)","Abandono de função (art. 323)","Exercício funcional ilegalmente antecipado ou prolongado (art. 324)","Violação do sigilo funcional (art. 325)","Violação do sigilo de proposta de concorrência (art. 326)","Inserção de dados falsos em sistema de informações (art. 313-A)","Modificação ou alteração não autorizada de sistema de informações (art. 313-B)","Contra Patente de Invenção","Contra os Desenhos Industriais","Contra as Marcas","Cometidos por Meio de Marca, Título de Estabelecimento e Sinal de Propaganda","Contra Indicações Geográficas e Demais Indicações","De Concorrência Desleal","de Responsabilidade","Contra a Economia Popular","de Abuso de Autoridade","Tráfico de Drogas e Condutas Afins","Posse de Drogas para Consumo Pessoal","Fabricação de Objeto Destinado a Produção de Drogas e Condutas Afins","Indução, Instigação ou Auxílio ao Uso de Drogas","Oferecimento de Drogas para Consumo Conjunto","Associação para a Produção e Tráfico e Condutas Afins","Financiamento ou Custeio de Produção ou Tráfico de Drogas","Colaboração com Grupo, Organização ou Associação Destinados à Produção ou Tráfico de Drogas","Prescrição Culposa de Drogas","Condução de Embarcação ou Aeronave sob Efeito de Drogas","Responsabilidade Criminal por Danos Nucleares","Contra o Serviço Postal e o Serviço de Telegrama","Contra a Segurança Nacional, a Ordem Política e Social","Contra o Sistema Financeiro Nacional","Resultante de Preconceito de Raça ou de Cor","Contra a Ordem Tributária","Contra a Ordem Econômica","Contra as Relações de Consumo","Representação caluniosa","Contra a Fauna","Contra a Flora","Da Poluição","Agrotóxicos (Lei 7.802/89)","Atividades Nucleares (Lei 6.453/77)","Caça (Lei nº 5.197/67)","Genética (Lei nº 8.974/95)","Pesca (Lei nº 5.197/67, Lei nº 7.643/87, Lei 7.679/88, DL 221/67)","ou Ocultação de Bens, Direitos ou Valores","Contra as Telecomunicações","Previstos na Lei de Estrangeiros","De Tortura","De Trânsito","Do Sistema Nacional de Armas","De Imprensa","Estatuto da criança e do adolescente","Contra o mercado de capitais","De genocídio","Sigilo Telefônico","Da Lei de licitações","Da Lei de remoção de órgãos e tecidos","Praticados contra os índios e a cultura indígena","Recusa, retardamento ou omissão de dados técnicos indispensáveis à propositura de Ação Civil Pública","Agrários","Relacionados à Comissão Parlamentar de Inquérito","Eleitorais","De Quebra de Sigilo Financeiro","Do Código Brasileiro de Telecomunicações","Estatuto do Idoso","Parcelamento do solo urbano","Falimentares","Militares","Contra a Administração da Justiça - Lei da Ação de Alimentos","Contravenções Penais","Análogo a Crime Tentado","Análogo a Crime Culposo","Usurpação de função pública (art. 328)","Resistência (art. 329)","Tráfico de influência (art. 332)","Corrupção ativa (art. 333)","Impedimento, perturbação ou fraude de concorrência (art. 335)","Inutilização de edital ou de sinal (art. 336)","Subtração ou inutilização de livro ou documento (art. 337, caput)","Desobediência (art. 330)","Desacato (art. 331)","Contrabando ou descaminho (art. 334)","Sonegação de contribuição previdenciária (art. 337-A e Lei 8.212/91)","Corrupção Ativa em Transação Comercial Internacional","Tráfico de Influência em Transação Comercial Internacional","Reingresso de estrangeiro expulso","Denunciação caluniosa","Comunicação falsa de crime ou de contravenção","Auto-acusação falsa","Falso testemunho ou falsa perícia","Coação no curso do processo","Exercício arbitrário das próprias razões","Fraude processual","Favorecimento pessoal","Favorecimento real","Exercício arbitrário ou abuso de poder","Fuga de pessoa presa ou submetida a medida de segurança","Evasão mediante violência contra a pessoa","Arrebatamento de preso","Motim de presos","Patrocínio infiel","Patrocínio simultâneo ou tergiversação","Sonegação de papel ou objeto de valor probatório","Exploração de prestígio","Violência ou fraude em arrematação judicial","Desobediência a decisão judicial sobre perda ou suspensão de direitos","Contratação de Operação de Crédito","Inscrição de Despesas Não Empenhadas","Assunção de Obrigação no Último Ano do Mandato ou Legislatura","Ordenação de Despesa Não Autorizada","Prestação de Garantia Graciosa","Não Cancelamento de Restos a Pagar","Aumento de Despesa com Pessoal no Último Ano do Mandato ou Legislatura","Oferta Pública ou Colocação de Títulos no Mercado","Abandono Material","Abandono Intelectual","Maus Tratos","Abuso Sexual","Exploração do Trabalho Infantil","Pobreza","Uso ou Tráfico de Drogas","Adoção Internacional","Adoção Nacional","Adoção de Criança","Adoção de Adolescente","Entrada e Permanência de Menores","Viagem ao Exterior","Trabalho do adolescente","Em comum / De fato","Anistia Política","Direitos Indígenas","Intervenção em Estado / Município","Indenização por Dano Ambiental","Erro Médico","Acidente de Trânsito","Declaração de Bagagem","Declaração de Trânsito Aduaneiro","Comercialização sem Restrições de Produtos Industrializados","Comercialização sem Restrições de Gêneros Alimentícios","Comercialização e/ou Utilização sem Restrições de Medicamentos","Funcionamento de Comércio de Derivados de Petróleo","Registro de Empresa","Funcionamento de Estabelecimentos Empresariais","Registro / Porte de arma de fogo","Registro de Aeronave","Inquérito / Processo / Recurso Administrativo","Ato Lesivo ao Patrimônio Artístico, Estético, Histórico ou Turístico","Dano ao Erário","Enriquecimento ilícito","Violação aos Princípios Administrativos","Inspeção Fitossanitária","Exportação/Vedações","Inspeção Sanitária de Origem Animal","Segurança e Medicina do Trabalho","Registro de Trabalho Portuário/Órgão de Gestão de Mão-de-Obra","Registro de Marcas, Patentes ou Invenções","Multas e demais Sanções","Interdição","Apreensão","Registro de Direito Autoral","Registro de Programa de Computador","Matrícula","Financiamento Público da Educação e/ou Pesquisa","Colação de Grau","Exigência de Estágio Profissionalizante","Transferência de Estudante","Freqüência às Aulas","Penalidades Disciplinares","Colisão de Horário","Vestibular","Mensalidades","Currículo Escolar","Ingresso no Curso Superior","Jubilamento de Aluno","Criação e/ou Autorização para funcionamento de Curso Superior","Omissão de Entrega de Notas","Diplomas/Certificado de Conclusão do Curso","Financiamento Privado da Educação e/ou Pesquisa","Residência Médica","Provão - Avaliação da Educação Superior","Sistemas de cotas - Lei 10.558/2002","Pós-Graduação","Mensalidade","Matrícula","Material Didático","Exame Nacional de Ensino Médio / ENEM","Penalidades Disciplinares","Transferência","Ensino Especial","Exame Supletivo","Merenda","Transporte","Educação Pré-escolar","Educação Profissionalizante","Repasse de Verbas do SUS","Reajuste da Tabela do SUS","Convênio Médico com o SUS","Ressarcimento ao SUS","Genética / Células Tronco","Sistema Nacional do Desporto","Fornecimento de Gás","Energia Elétrica","Transporte Terrestre","Transporte Aéreo - Aeroporto","Radiodifusão","Loterias/Sorteio","Telefonia","Transporte Aquaviário","Serviço Postal","Jogos de Bingo e/ou Caça-níqueis","Água e/ou Esgoto","Recolhimento e Tratamento de Lixo","Defensoria Pública","Locação / Permissão / Concessão / Autorização / Cessão de Uso","Terreno de Marinha","Taxa de Ocupação","Foro / Laudêmio","Terras Devolutas","Águas Públicas","Bloqueio de Valores de Contas Públicas","Alienação","Ocupação","Reintegração de Posse","Taxa de Ocupação","Terreno Aldeado","Restituição de área - FUNAI","Demarcação","Recursos Minerais","Espaço Aéreo","Patrimônio Histórico / Tombamento","Ordenação da Cidade / Plano Diretor","Revogação/Concessão de Licença Ambiental","Revogação/Anulação de multa ambiental","Flora","Fauna","Transgênicos","Agrotóxicos","Unidade de Conservação da Natureza","Gestão de Florestas Públicas","Desapropriação","Desapropriação por Utilidade Pública / DL 3.365/1941","Desapropriação por Interesse Social Comum / L 4.132/1962","Desapropriação por Interesse Social para Reforma Agrária","Desapropriação Indireta","Retrocessão","Vistoria","Servidão Administrativa","Ocupação Temporária","Limitação Administrativa","Privatização","Restituição de área","Direito de Preempção","Desapropriação de Imóvel Urbano","Requisição de Bem Particular","Política de Preço Mínimo","Controle de Preços","Controle de Abastecimento","Desembaraço Aduaneiro","Regime ou Certificado de Origem MERCOSUL","Veículo de Turista Comunitário MERCOSUL","PROAGRO","Venda Casada","Preços Predatórios","Acordo de Exclusividade","Cartel","Atos de Concentração","Moeda Estrangeira","Cruzados Novos / Bloqueio","Poupança","Liberação de Conta","Juros Progressivos","Localização de Contas","Atualização de Conta","Liberação de Conta","Registro Profissional","Multas e demais Sanções","Questões Funcionais","Exame da Ordem OAB","Eleições","Anuidades OAB","Exercício Profissional","Liberação de Depósitos de Instituição Liquidanda","Normatizações","Remuneração de Ativos Retidos","Cooperativas de Crédito","Liquidação extrajudicial","Fundo de Participação dos Municípios","Seguro desemprego","Apólices da Dívida Pública","Títulos da Dívida Pública","Títulos da Dívida Agrária","Remuneração","Afastamento","Processo Disciplinar / Sindicância","Promoção","Remuneração","Afastamento","Processo Disciplinar / Sindicância","Promoção","Afastamento do Cargo","Remuneração","Recondução","Prestação de Contas","Indisponibilidade de Bens","Afastamento do Cargo","Remuneração","Recondução","Prestação de Contas","Indisponibilidade de Bens","Afastamento do Cargo","Recondução","Prestação de Contas","Indisponibilidade de Bens","Estabilidade","Enquadramento","Recondução","Acumulação de Cargos","Reintegração","Direito de Greve","Transferência","Remoção","Regime Previdenciário","Anistia Administrativa","Reversão","Redistribuição","Disponibilidade / Aproveitamento","Lotação","Promoção / Ascensão","Readaptação","Estágio Probatório","Posse e Exercício","Exoneração","Complementação de Benefício/Ferroviário","Assistência à Saúde","Assistência Pré-escolar","Auxílio-Natalidade","Auxílio-Reclusão","Auxílio-Funeral","Salário-Família","Provisória","Concessão","Restabelecimento","Invalidez Permanente","Compulsória","Voluntária","Dirigente Sindical","Licença Capacitação (Aperfeiçoamento Profissional)","Doença em Pessoa da Família","Tratamento da Própria Saúde","Gestante / Adotante / Paternidade","Interesse Particular","Serviço Militar","Atividade Política","Casamento","Luto","Amamentação","Doação de Sangue","Alistamento / Serviço Eleitoral","Acompanhamento de Cônjuge ou Companheiro","Exercício em Outro Município","Licença por Acidente em Serviço","Averbação / Contagem de Tempo Especial","Averbação / Contagem Recíproca","Demissão ou Exoneração","Advertência","Suspensão","Responsabilidade Civil do Servidor Público / Indenização ao Erário","Programa de Desligamento Voluntário (PDV)","Jornada de Trabalho","Gratificações da Lei 8.112/1990","Gratificação de Incentivo","Adicional de Periculosidade","Adicional de Fronteira","Abono Pecuniário (Art. 78 Lei 8.112/1990)","Gratificação Incorporada / Quintos e Décimos / VPNI","Descontos Indevidos","Teto Salarial","Diárias e Outras Indenizações","Plano de Classificação de Cargos","Data Base","Adicional de Horas Extras","Auxílio-Alimentação","Gratificações de Atividade","Auxílio-transporte","Revisão Geral Anual (Mora do Executivo - inciso X, art. 37, CF 1988)","Adicional de Serviço Noturno","Adicional de Produtividade","Gratificação Natalina/13º salário","Irredutibilidade de Vencimentos","Piso Salarial","Índice da URV fev/1989","Índice de 84,32% março/1990","Índice de 45% Lei 8.237/1991","Índice de 28,86% Lei 8.622/1993 e 8.627/1993","Reajuste da Lei 8.270/1991","Índice do IPC junho/1987","Índice de 10,87% Lei 10.192/2001","Índice de 47,94% Lei 8.676/1993","Inclusão de Dependente","Ingresso e Concurso","Curso de Formação","Reintegração","Estabilidade","Anistia Política","Adidos, Agregados e Adjuntos","Serviço Militar Obrigatório","Ex-combatentes","Promoção","Serviço Militar dos Profissionais da Saúde","Corpo Feminino","Gratificações e Adicionais","Férias","Indenizações Regulares","Índice de 28,86% LL 8.622/1993 e 8.627/1993","Índice da URV Lei 8.880/1994","Índice do IPC  junho/1987","Índice de 84,32% IPC março/1990","Índice da URP abril e maio/1988 DL 2.425/1988","Reforma","Licenças","Agregação","Reserva Remunerada","Transferência ex-officio para reserva","Tempo de Serviço","Transferência para reserva","Assistência Médico-Hospitalar","Licença Prêmio","Licenciamento","Concessão","Restabelecimento","Provisória","Advertência / Repreensão","Impedimento / Detenção / Prisão","Licenciamento / Exclusão","Suspensão","Reserva de Vagas para Deficientes","Inscrição / Documentação","Limite de Idade","Exigência de Prática Forense","Prova de Títulos","Exame de Saúde e/ou Aptidão Física","Curso de Formação","Exame Psicotécnico / Psiquiátrico","Anulação e Correção de Provas / Questões","Escolaridade","Classificação e/ou Preterição","Anulação","Prazo de Validade","Condições Especiais para Prestação de Prova","Modalidade / Limite / Dispensa / Inexigibilidade","Habilitação / Registro Cadastral / Julgamento / Homologação","Edital","Revogação","Sanções Administrativas","Recursos Administrativos","Convênio","Adjudicação","Ambiental","Sanitárias","Metrológica","Profissional","Segurança e/ou Medicina do Trabalho","Taxa de Ocupação / Laudêmio / Foro","Cessão de créditos não-tributários","Contribuição Sindical","Eleições Sindicais","Registro Sindical","Dilação e Extensão de Base Territorial","Filiação","Contrato Temporário de Mão de Obra L 8.745/1993","Admissão / Permanência / Despedida","Quebra de Sigilo Bancário / Fiscal / Telefônico","Criação / Instalação / Prosseguimento / Encerramento","Limites dos Poderes de Investigação","Limites do Objeto","CNH - Carteira Nacional de Habilitação","Liberação de Veículo Apreendido","Licenciamento de Veículo","Pagamento Atrasado / Correção Monetária","Anulação","Suspensão","Rescisão","Prorrogação","Termo Aditivo","Penalidades","Execução Contratual","Equilíbrio Financeiro","Erro Médico","Acidente de Trânsito","Lei de Imprensa","Direito de Imagem","Dano Ambiental","Erro Médico","Acidente de Trânsito","Lei de Imprensa","Direito de Imagem","Esbulho / Turbação / Ameaça","Imissão","Aquisição","Perda da Propriedade","Adjudicação Compulsória","Divisão e Demarcação","Reivindicação","Retificação de Área de Imóvel","Alienação Judicial","Acessão","Usucapião Especial (Constitucional)","Usucapião Extraordinária","Usucapião Ordinária","Usucapião Especial Coletiva","Direito de Vizinhança","Condomínio","Administração","Alteração de Coisa Comum","Assembléia","Despesas Condominiais","Direitos / Deveres do Condômino","Vaga de garagem","Incorporação Imobiliária","Propriedade Resolúvel","Propriedade Fiduciária","Enfiteuse","Servidão","Usufruto","Superfície","Uso","Habitação","Rural - Agrícola/Pecuário","Industrial / Mercantil","Direitos e Títulos de Crédito","Veículos","Legal","Hipoteca","Anticrese","Promessa de Compra e Venda","Subsídios","Programa de Computador","Usucapião da L 6.969/1981","Crédito Rural","Erro Médico","Acidente de Trânsito","Variação Cambial","Maus Tratos","Exposição ou Abandono de Recém Nascido","Responsabilidade Civil do Militar - Indenização ao Erário","Utilização de Embrião Humano em Desacordo com a Legislação","Prática de engenharia genética em célula germinal, zigoto ou embrião humanos","Clonagem Humana","Despenalização / Descriminalização","Livros / Jornais / Periódicos","Entidades Sem Fins Lucrativos","Partidos Políticos","Imunidade Recíproca","ICMS / Incidência Sobre o Ativo Fixo","Incidência sobre Auxílio-condução","Incidência sobre Auxílio-creche","Taxa de Limpeza Pública","Taxa de Iluminação Pública","Taxa de Coleta de Lixo","Taxa de Prevenção e Combate a Incêndio","Taxa de Licenciamento de Estabelecimento","Capacidade Tributária","Competência Tributária","Depósito Judicial","Constitucionalidade do artigo 4º da LC 118/05","Constitucionalidade do artigo 46 da Lei 8212/91","Decretação de Ofício","Arquivamento Administrativo - Crédito de Pequeno Valor","Despacho de Citação","Constitucionalidade do artigo 45 da Lei 8212/91","Exclusão - ICMS","Exclusão - IPI","Exclusão - Receitas Provenientes de Exportação","Quebra de Sigilo Bancário","PIS - Importação","COFINS - Importação","Compensação de Prejuízo","Contribuição Sindical Rural","Contribuição Sindical Rural","Aposentadoria por Invalidez Acidentária","Rural","FGTS","Acidente de Trabalho","Termo de Adesão da LC 110/2001","Rescisão / Resolução","Capitalização / Anatocismo","Limitação de Juros","Proteção da Intimidade e Sigilo de Dados","Vícios de Construção","Juros Compensatórios","Resgate de Contribuição","Inclusão de Dependente","Desconto em folha de pagamento","Remuneração Mínima","Multa","Auxílio-invalidez","Cobrança indevida de ligações","Quesitos","Impedimento","Suspeição","Condicional do processo","Parcelamento de crédito tributário","Competência do MP","Cerceamento de Defesa","Quebra do Sigilo Bancário","Quebra do Sigilo Fiscal","Quebra do Sigilo Telefônico","Trancamento","Abolitio Criminis","Princípio da Insignificância","Legítima Defesa","Estado de Necessidade","Estrito Cumprimento do Dever Legal","Inexigibilidade de Conduta Diversa","Prescrição","Perdão","Anistia","Indulto","Pagamento ou Parcelamento do Crédito Tributário","Substituição da Pena","Transação","Desaforamento","Prisão Temporária","Regime inicial","Suspensão condicional da pena","Livramento condicional","Remição","Acumulação de Proventos","Prisão Decorrente de Sentença Condenatória","Penas do Código Penal Militar","Perda da Função Pública","Perda de Bens e Valores","Inconstitucionalidade Material","Processo Legislativo","Competência dos Juizados Especiais","Competência da Justiça do Trabalho","Competência da Justiça Federal","Competência da Justiça Estadual","Honorários Advocatícios","Honorários Advocatícios em Execução Contra a Fazenda Pública","Honorários Advocatícios em FGTS","Custas","Suspeição","Impedimento","Incidência sobre Abono de Permanência","Abono de Permanência","Regularidade Formal","Fac-símile","Protocolo Integrado / Descentralizado","Obrigação de Fazer / Não Fazer","Requisição de Pequeno Valor - RPV","Quinto Constitucional","Imissão na Posse","Busca e Apreensão","Sequestro de Verbas Públicas","Fracionamento","Crédito Complementar","Aposentadoria / Pensão Especial","Fato Superveniente ao Término do Prazo para Impugnação","Juros","Correção Monetária","Multa Cominatória / Astreintes","Taxa SELIC","Correção da Tabela","IPMF - Imposto Provisório sobre Movimentação Financeira","Questão Prejudicial","Tribunal de Contas","Cargo em Comissão","Professor","Procuradores de Órgãos / Entidades Públicos","Serventuários da Justiça","Extensão de Vantagem aos Inativos","Contagem em Dobro","Pagamento em Pecúnia","Índice de 3,17%","Índice de 11,98%","Subteto Salarial","Gratificações Municipais Específicas","Gratificação de Encargos Especiais - GEE","Gratificação de Estímulo à Produção Individual - GEPI","Gratificação Estadual - AM","Adicional de Etapa Alimentar","Gratificação de Suporte às Atividades Escolares - GSAE","Gratificação por Trabalho Educacional - GTE","Gratificação de Suporte Administrativo - GASA","Gratificação-Prêmio","Gratificação Complementar de Vencimento","Gratificação de Desempenho de Atividade Técnico Administrativa - GDATA","Gratificação de Desempenho de Atividade Tributária - GDAT","Gratificação de Desempenho de Atividade de Fiscalização Agropecuária - GDAFA","Gratificação de Desempenho de Atividade Jurídica - GDAJ","Gratificação de Atividade - GATA","Gratificação de Incentivo à Docência - GID","Gratificação de Produção Suplementar - GPS","Gratificação Eleitoral","Gratificação de Inatividade","Gratificação Extraordinária - GE","Gratificação de Desempenho de Função - GADF","Adicional de Desempenho","Adicional de Sexta-Parte","Verba de Representação","Extensão de Vantagem aos Inativos","Nulidade - Não Observância da Reserva de Plenário","Ausência de Interesse Processual","Ausência de Pressupostos de Constituição e Desenvolvimento","Sobrestamento","Criação do Partido Político","Registro do Partido Político","Extinção do Partido Político","Filiação partidária","Convenção para escolha de candidatos","Inscrição fraudulenta","Calúnia","Difamação","Injúria","Captação ilícita de votos ou corrupção eleitoral","Aliciamento eleitoral","Falso eleitoral","Desobediência à determinações da Justiça Eleitoral","Boca de urna","Indução à inscrição indevida","Fidelidade partidária","Plebiscito","Referendo popular","Sistema Majoritário / Proporcional","fixação","aumento","redução","Domicílio eleitoral","Isenção do dever eleitoral","Justificação de ausência de voto","Cancelamento da inscrição de eleitor","Desincompatibilização do candidato","Inelegibilidade","Reeleição","Cassação do registro da candidatura","Anistia","Propaganda eleitoral","Direito de resposta","Prestação de contas","Pesquisa eleitoral","Abuso do poder econômico","Abuso do poder político (uso indevido dos meios de comunicação)","Uso de bem público","Sistema eletrônico de votação","Sigilo na votação","Apuração/recontagem de votos","Nulidade da votação","Quociente eleitoral","Transporte em dia de eleição","Boca de urna","Cassação do diploma","Sustação da diplomação","Prestação de contas","Declaração de inelegibilidade","Impugnação de mandato eletivo","Perda do mandato","Recondução","Suspensão de mandato","Convocação de suplente","Renúncia","Afastamento do cargo","Cassação de mandato","Por cumulação de cargos","Por condenação criminal","Por suspensão dos direitos políticos","Desordem eleitoral","Falsificação de documento público para fins eleitorais","Uso de documento falso","Obtenção de documento falso","Aliciamento de eleitores","Divulgação de pesquisa de fraudulenta","Impedimento à fiscalização das pesquisas","Publicação de pesquisa irregular","Uso de símbolos governamentais em propaganda eleitoral","Coação eleitoral","Aliciamento violento de eleitores","Aliciamento comercial de eleitores","Ocultação de urna","Motim","Pederastia ou outro ato de libidinagem","Estelionato","Incêndio","Peculato","Concussão","Corrupção ativa","Prevaricação","Deserção","Lesão levíssima","Difamação","Injúria","Homicídio simples","Homicídio qualificado","Leve","Grave","Lesões qualificadas pelo resultado","Roubo simples","Roubo qualificado","Latrocínio","Auditores Fiscais","Comissão de Permanência","Prescrição por Médico não vinculado ao SUS","Gratificação de Desempenho de Atividade de Seguridade Social e do Trabalho - GDASST","Gratificação de Desempenho de Atividade de Ciência e Tecnologia - GDACT","Alimentos","Depositário Infiel","Alienação Fiduciária","Proibição de Privilégio Fiscal às Empresas Públicas e Sociedades de Economia Mista","Ausência de Legitimidade para propositura de Ação Civil Pública","Crimes Hediondos","Cerceamento de Defesa","Ausência de Publicidade","Ausência de Fundamentação","Criação / Extinção / Reestruturação de Orgãos ou Cargos Públicos","Parcela Incontroversa","Cancelamento / Duplicidade de CPF","Não Cumulatividade","Nao Cumulatividade","Não Cumulatividade","Não Cumulatividade","Base de Cálculo","Policiais Civis","Limite de Renda Familiar","Cumprimento Provisório de Sentença","Nepotismo","Fruição / Gozo","Indenização / Terço Constitucional","Liquidação Parcelada","Nulidade do Decreto que autoriza a desapropriação","Ausência de Cobrança Administrativa Prévia","Inscrição Indevida no CADIN","Efeitos da Declaração de Inconstitucionalidade","Vício Formal do Julgamento","Habeas Corpus - Cabimento","Medicamento / Tratamento / Cirurgia de Eficácia não comprovada","Base de Cálculo","Abuso de Poder","Competência dos Juizados Especiais","Competência da Justiça Federal","Competência da Justiça Estadual","Competência por Prerrogativa de Função","Competência por Prerrogativa de Função","Excesso de prazo para instrução / julgamento","Incidência sobre desconto incondicional","Prisão Domiciliar / Especial","Passe livre em transporte","Regressão de Regime","Transferência de Preso","Execução Penal Provisória - Cabimento","Imunidade em razão de função, ofício ou profissão","Transferência de Unidade","Indisponibilidade / Seqüestro de Bens","Busca e Apreensão de Bens","Aborto provocado pela gestante ou com o seu consentimento","Aborto provocado por terceiro","Aborto qualificado","Restituição de Criança, Convenção de Haia 1980","Cobrança de Multa Moratória de Massa Falida","Cobrança de Juros Moratórios de Massa Falida","Depósito Elisivo","Prova Ilícita","Competência do Órgão Fiscalizador","Criação","Desmembramento","Desclassificação","Guarda com genitor ou responsável no exterior","Citação","Intimação / Notificação","Depoimento","Viagem Nacional","Expurgos Inflacionários / Planos Econômicos","Correção Monetária de Diferenças Pagas em Atraso","Princípio da Seletividade","Violência Doméstica Contra a Mulher","Violência Doméstica Contra a Mulher","Crime / Contravenção contra Criança / Adolescente","Crime / Contravenção contra Idoso","Fato Atípico","Criação de Dotação Orçamentária","Repasse de Duodécimos","Repasse de Verbas Públicas","Crédito Suplementar","Exclusão - Receitas Transferidas a outras Pessoas Jurídicas","Lavagem ou Ocultação de Bens, Direitos ou Valores Oriundos de Corrupção","Lavagem ou Ocultação de Bens, Direitos ou Valores Oriundos de Corrupção","Lavagem ou Ocultação de Bens, Direitos ou Valores Oriundos de Corrupção","Crimes contra a Ordem Tributária praticado por Funcionário Público","Crime contra a administração ambiental","Tráfico Ilícito de Drogas praticado por Funcionário Público","Má -gestão praticada por Prefeitos e Vereadores","Má-gestão praticada por Prefeitos e Vereadores","Corrupção Praticada por Prefeitos e Vereadores","Adulteração de Sinal Identificador de Veículo Automotor por Funcionário Público","Petrechos de Falsificação de Papéis Públicos por Funcionário Público","Petrechos para falsificação de moeda por Funcionário Público","Falsidade ideológica praticada por Funcionário Público","Falsificação de documento público praticada por Funcionário Público","Falsificação de documento Público praticada por Funcionário Público","Conversão em Agravo Retido","Cobrança de Aluguéis - Sem despejo","Consignação de Chaves","Falsificação do Selo ou Sinal Público Praticado por Funcionário Público","Falsificação de Papéis Públicos Praticado por Funcionário Público","Crime de Desvio e/ou Circulação de Moeda Não Autorizada","Crime de Moeda Falsa Praticado por Funcionário Público","Competência da Justiça Militar da União","Competência da Justiça Militar dos Estados","Menagem","Competência da Justiça Militar da União","Competência da Justiça Militar Estadual","Cassação de licença para direção de veículo motorizado","Confisco","Exílio local","Interdição de estabecimento  ou associação","Proibição de frequentar determinados locais","Suspensão do exercício do posto, graduação, cargo ou função","Representação do Corregedor","Requerimento da Parte","Irregularidade na composição do Conselho","Crime Culposo","Crime em tempo de guerra","Crime Tentado","Livramento Condicional","Suspensão Condicional da Pena","Abolitio criminis","Princípio da Insignificância","Exclusão das Forças Armadas","Inabilitação para o exercício de Função Pública","Incompatibilidade para o Oficialato","Indignidade para o Oficialato","Perda da Função Pública","Perda da Graduação das Praças","Perda do Posto e da Patente","Suspensão do pátrio poder, tutela ou curatela","Suspensão dos direitos políticos","Anistia","Indulto","Morte do Agente","Prescrição","Reabilitação","Ressarcimento do Dano","Estado de Necessidade","Estado de Necessidade exculpante","Estrito Cumprimento do Dever Legal","Exercício Regular do Direito","Inexigibilidade de Conduta Diversa","Legítima Defesa","Perda em favor da Fazenda Pública","Reparação do Dano","Conversão da Pena","Substituição da Pena","Criação ou simulação de incapacidade física","Favorecimento a convocado","Insubmissão","Insubmissão privilegiada","Substituição de convocado","Exercício de comércio por oficial","Concerto para deserção","Deserção","Deserção - casos assimilados","Deserção especial","Deserção por evasão ou fuga","Deserção privilegiada","Deserção qualificada","Favorecimento a desertor","Omissão de Oficial","Abandono de posto","Descumprimento de missão","Dormir em serviço","Embriaguez em serviço","Omissão de eficiência da força","Omissão de providências para evitar danos","Omissão de providências para salvar comandados","Omissão de socorro","Retenção indevida","Alteração de Limites","Aposição, supressão ou alteração de marca","Usura pecuniária","Chantagem","Extorsão","Extorsão indireta","Extorsão mediante sequestro","Latrocínio","Roubo","Roubo qualificado","Receptação","Furto","Furto de uso","Furto qualificado","Abuso de pessoa","Estelionato","Dano atenuado","Dano em aparelhos e instalações","Dano em navio de guerra ou mercante em serviço militar","Dano material ou aparelhamento de guerra","Dano qualificado","Dano simples","Desaparecimento,consunção ou extravio","Apropriação de coisa achada","Apropriação de coisa havida acidentalmente","Apropriação indébita","Abuso de radiação","Desabamento ou desmoronamento","Difusão de epizootia ou praga vegetal","Embriaguez ao volante","Emprego de gás tóxico ou asfixiante","Explosão","Fatos que expõem a perigo aparelhamento militar","Fuga após acidente de trânsito","Incêndio","Inundação","Perigo de inundação","Perigo resultante de violação de regra de trânsito","Subtração, ocultação ou inutilização de material de socorro","Arremesso de projétil","Atentado contra serviço de utilidade militar","Atentado contra transporte","Atentado contra viatura ou outro meio de transporte","Interrupção ou perturbação de serviço ou meio de comunicação","Perigo de desastre ferroviário","Corrupção ou poluição de água potável","Envenenamento com perigo extensivo","Epidemia","Fornecimento de substância alterada","Fornecimento de substancia nociva","Omissão de notificação de doença","Receita Ilegal","Tráfico, posse ou uso de entorpecente ou substância de efeito similar","Atos de jurisdição indevida","Consecução de notícia, informação ou documento para fim de espionagem","Desenho ou levantamento de plano ou planta de local militar","Entendimento para empenhar o Brasil à neutralidade ou à guerra","Entendimento para gerar conflito ou divergência com o Brasil","Hostilidade contra país estrangeiro","Hostilidade qualificada pelo resultado","Penetração com o fim de espionagem","Provocação a país estrangeiro","Revelação de notícia, informação ou documento","Revelação de notícia, informação ou documento, com o fim de espionagem militar","Sobrevôo em local interdito","Tentativa contra a soberania do Brasil","Turbação de objeto ou documento","Violação de território estrangeiro","Ato obsceno","Escrito ou objeto obsceno","Abandono de pessoa","Abandono de pessoa seguida de morte ou lesão grave","Maus tratos","Maus tratos seguido de morte ou lesão grave","Lesão grave","Lesão leve","Lesão levíssima","Lesão privilegiada","Lesão seguida de morte","Participação em rixa","Homicídio","Homicidio qualificado","Homicido privilegiado","Provocação direta ou auxílio a suicídio","Provocação indireta ao suicídio","Genocídio","Atentado violento ao pudor","Corrupção de menores","Estupro","Ato Libidinoso","Ameaça","Constragimento ilegal","Desafio para duelo","Divulgação de segredo","Sequestro ou cárcere privado","Violação de correspondência","Violação de domicílio","Violação de recato","Violação de segredo profissional","Calúnia","Difamação","Injúria","Injuria real","Ofensa às forças armadas","Violência contra militar de serviço","Violência contra superior","Abuso de requisição militar","Assunção de comando sem ordem ou autorização","Conservação ilegal de comando","Ofensa aviltante a inferior","Operação militar sem ordem superior","Ordem arbitrária de invasão","Rigor excessivo","Uso indevido de uniforme, distintivo ou insígnia militar por qualquer pessoa","Uso indevido por militar de uniforme, distintivo ou insígnia","Violência contra inferior","Resistência mediante ameaça ou violência","Conspiração","Motim","Omissão de lealdade militar","Organização de grupo para prática de violência","Revolta","Oposição a ordem de sentinela","Publicação ou crítica indevida","Recusa a obediência","Reunião ilícita","Amotinamento","Arrebatamento de preso ou internado","Evasão de preso ou internado","Fuga de preso ou internado","Aliciação para motim ou revolta","Apologia de fato criminoso ou do seu autor","Incitamento","Despojamento desprezível","Desrespeito a comandante, oficial-general ou oficial de serviço","Desrespeito a símbolo nacional","Desrespeito a superior","Peculato","Peculato mediante aproveitamento ou erro de outrem","Peculato-furto","Certidão ou atestado ideologicamente falso","Cheques sem fundos","Falsa identidade","Falsidade ideológica","Falsificação de documento","Supressão de documento","Uso de documento falso","Uso de documento pessoal alheio","Desacato a assemelhado ou funcionário","Desacato a militar","Desacato a superior","Desobediência","Ingresso clandestino","Impedimento, perturbação ou fraude de concorrencia","Inutilização de edital ou de sinal oficial","Subtração ou inutilização de livro, processo ou documento","Tráfico de influência","Usurpação de função","Abandono de cargo","Abuso de confiança ou boa fé","Aplicação ilegal de verba ou dinheiro","Condescendência criminosa","Exercício funcional ilegal","Extravio, sonegação ou inutilização de livro ou documento","Inobservancia de lei, regulamento ou instrução","Não inclusão de nome em lista","Obstáculo à hasta pública, concorrência ou tomada de preços","Patrocínio indébito","Prevaricação","Violação de sigilo de proposta de concorrência","Violação de sigilo funcional","Violação do dever funcional com o fim de lucro","Violação ou divulgação indevida de correspondência ou comunicação","Violência arbitrária","Corrupção ativa","Corrupção passiva","Participação ilícita","Concussão","Desvio","Excesso de exação","Auto-acusação falsa","Coação","Comunicação falsa de crime","Corrupção ativa de testemunha, perito ou intérprete","Denunciação caluniosa","Desacato","Desobediência a decisão judicial","Desobediência a decisão sobre perda ou suspensão de atividade ou direito","Exploração de prestígio","Falso testemunho ou falsa perícia","Favorecimento pessoal","Favorecimento real","Inutilização,sonegação ou descaminho de material probante","Publicidade opressiva","Recusa de função na Justiça Militar","Advertência","Internação com atividades externas","Internação sem atividades externas","Liberdade assistida","Obrigação de reparar o dano","Prestação de serviços à comunidade","Semiliberdade","Crimes contra portadores de deficiência","Conflito fundiário coletivo rural","Conflito fundiário coletivo urbano","Crime/contravenção decorrente de conflito fundiário coletivo","Violação sexual mediante fraude","Estupro de vulnerável","Satisfação de lascívia mediante presença de criança ou adolescente","Favorecimento da prostituição ou outra forma de exploração sexual de vulnerável","Favorecimento da prostituição ou outra forma de exploração sexual","Tráfico internacional de pessoa para fim de exploração sexual","Tráfico interno de pessoa para fim de exploração sexual","Intervenção Indevida de Autoridade Junto à Mesa Receptora","Não Observância de Chamamento de Eleitores para Votar","Práticas Irregulares que Determinam a Anulação da Votação","Votação de Eleitor na Seção Eleitoral na qual Não Está Inscrito ou Autorizado a Votar","Contagem de Votos de Eleitores em Relação aos quais Houve Impugnação","Fabricação, Aquisição, Fornecimento, Subtração ou Guarda de Materiais de Uso da Justiça Eleitoral","Inscrição Fraudulenta","Induzimento à Inscrição de Eleitor em Infração às Normas Legais","Inscrição Fraudulenta Efetivada pelo Juiz","Negativa ou Retardamento de Inscrição sem Fundamento Legal","Perturbação ou Impedimento de Alistamento","Promoção de Desordem nos Trabalhos Eleitorais","Majoração de Preços de Utilidade e Serviços Necessários à Realização das Eleições","Ocultação, Sonegação ou Recusa de Fornecimento de Utilidades, Alimentos e Transporte no Dia da Eleição","Retardamento ou Não-Publicação de Decisões, Citações ou Intimações da Justiça Eleitoral","Não-Apresentação de Denúncia ou Não Execução de Sentença Penal Condenatória no Prazo Legal","Não Cumprimento, pela Autoridade Judiciária, do Dever de Representar Contra o Órgão do Ministério que Não Oferece Denúncia no Prazo Legal","Recusa ou Abandono do Serviço Eleitoral","Não Cumprimento dos Deveres Impostos pelo Código Eleitoral à Autoridade Judiciária e aos Funcionários da Justiça Eleitoral","Utilização de Prédios ou Serviços de Repartições Públicas para Beneficiar Partido ou Organização de Caráter Político","Desobediência a Ordens ou Instruções da Justiça Eleitoral","Descumprimento do Dever de Informar o Número, Espécie e Lotação de Veículos e Embarcações","Desatendimento à Requisição de Veículo ou Embarcação","Descumprimento da Proibição de Fornecimento de Transporte ou Refeições a Eleitores","Estupro de Vulnerável","Favorecimento de Prostituição ou Outra Forma de Exploração Sexual","Favorecimento de Prostituição ou Outra Forma de Exploração Sexual de Vulnerável","Satisfação de Lascívia Mediante Presença de Criança ou Adolescente","Tráfico Interno de Pessoa para Fim de Exploração Sexual","Violação Sexual Mediante Fraude","Obstamento do Transporte e Fornecimento de Alimentos a Eleitores pela Justiça Eleitoral","Utilização de Veículos e Embarcações Públicas em Campanha Eleitoral","Dano a Equipamento Eleitoral","Omissão na Expedição de Boletim de Apuração","Omissão no Recolhimento das Cédulas Apuradas, no Fechamento e Lacração da Urna","Alteração do Resultado de Votação nos Mapas ou Boletins de Apuração","Omissão no Recebimento e Registro de Protestos, bem como, de sua Remessa à Instância Superior","Falsificação ou Alteração de Documento Público para Fins Eleitorais","Falsidade Ideológica","Reconhecimento Indevido de Firma ou Letras para Fins Eleitorais","Uso de Documento Falso para Fins Eleitorais","Obtenção de Documento Falso para Fins Eleitorais","Omissão da Entrega do Boletim de Urna","Obtenção de Acesso a Sistema de Dados Eleitorais","Destruição de Dados Eleitorais","Cerceamento da Fiscalização Partidária","Recusa de Entrega de Boletim","Divulgação de Fatos Inverídicos na Propaganda Eleitoral","Calúnia na Propaganda Eleitoral","Divulgação de Falsa Imputação","Difamação na Propaganda Eleitoral","Injúria na Propaganda Eleitoral","Injúria Eleitoral Violenta","Inutilização, Alteração ou Perturbação de Propaganda","Impedimento do Exercício da Propaganda","Utilização de Organização Comercial para Propaganda ou Aliciamento de Eleitores","Realização de Propaganda Eleitoral em Língua Estrangeira","Participação de Pessoa Não Detentora de Direitos Políticos em Atividades Partidárias e de Propaganda Eleitoral","Autorização de Transmissão de Pessoa Não Detentora de Direitos Políticos em Atividades Partidárias e de Propaganda Eleitoral","Pesquisa Fraudulenta","Uso, na Propaganda Eleitoral, de Símbolo de Órgãos de Governo","Retenção de Título Eleitoral ou do Comprovante de Alistamento Eleitoral","Impedimento ou Embaraço ao Exercício do Sufrágio","Prisão ou Detenção do Eleitor, Membro de Mesa Receptora, Fiscal, Delegado de Partido ou Candidato, Fora das Hipóteses Legais Permitidas","Corrupção Eleitoral","Coação Visando a Obtenção de Voto ou a sua Abstenção","Violência ou Grave Ameaça Visando a Obtenção de Voto ou a sua Abstenção","Concentração de Eleitores para Embaraçar ou Fraudar o Exercício do Voto","Fornecimento ao Eleitor de Cédula Oficial Já Assinalada ou Marcada","Fornecimento de Cédula Eleitoral Rubricada a Destempo","Votação Múltipla ou Realizada em Lugar de Outrem","Violação do Sigilo do Voto","Violação do Sigilo da Urna ou dos Invólucros","Destruição, Supressão ou Ocultação de Urna Contendo Votos ou Documentos Eleitorais","Destruição de Relação de Candidatos","Uso Proibido de Alto-Falantes","Arregimentação de Eleitor ou Boca de Urna","Divulgação de Propaganda Eleitoral no Dia da Eleição","Retenção de Título ou Comprovante de Alistamento Eleitoral","Subscrição de Mais de Uma Ficha de Filiação Partidária","Subscrição de Mais de Uma Ficha de Filiação Partidária","Inscrição Simultânea em Dois ou Mais Partidos","Coleta de Assinatura em Mais de Uma Ficha de Registro Partidário","Preterição à Prioridade Postal","Arguição de Ineligibilidade Temerária ou de Má-Fé","Não-Acesso dos Partidos aos Dados Relativos às Pesquisas Eleitorais","Irregularidades dos Dados Publicados em Pesquisas Eleitorais","Conduta Vedada a Agente Público","Improbidade Administrativa","Descumprimento de Prazos Eleitorais","Garantia de Autoridade de Decisão do Tribunal","Afastamento de Magistrado","Designação de Juiz Eleitoral","Lista Tríplice","Concurso Público","Remoção de Servidor","Requisição de Servidor","Requisição de Servidor - Renovação de Requisição de Servidor","Requisição de Força Federal","Alistamento Eleitoral - Cancelamento","Alistamento Eleitoral - Domicílio Eleitoral","Alistamento Eleitoral - Duplicidade/Pluralidade","Alistamento Eleitoral - Exclusão","Alistamento Eleitoral - Inscrição Eleitoral","Impugnação - Inscrição Eleitoral","Impugnação - Transferência de Domicílio Eleitoral","Condição de Elegibilidade - Alistamento Eleitoral","Condição de Elegibilidade - Domicílio Eleitoral na Circunscrição","Condição de Elegibilidade - Idade Mínima","Condição de Elegibilidade - Filiação Partidária","Condição de Elegibilidade - Nacionalidade Brasileira","Condição de Elegibilidade - Pleno Exercício dos Direitos Políticos","Condição de Elegibilidade - Quitação Eleitoral","Direito de Resposta","Identificação Numérica de Candidato","Inelegibilidade - Abuso do Poder Econômico ou Político","Inelegibilidade - Analfabetismo","Inelegibilidade - Condenação Criminal por órgão colegiado ou Transitada em Julgado","Inelegibilidade - Declaração de Indignidade ou de Incompatibilidade com o Oficialato","Inelegibilidade - Desincompatibilização","Inelegibilidade - Inalistável","Inelegibilidade - Parentesco","Inelegibilidade - Reeleição","Inelegibilidade - Rejeição de Contas Públicas","Inelegibilidade - Representação ou Ação de Investigação Judicial Eleitoral Jugada Procedente pela Justiça Eleitoral","Inelegibilidade - Terceiro Mandato","Inelegibilidade - Vida Pregressa","Inelegibilidade - Vínculo Afetivo","Inelegibilidade - Vínculo Conjugal","Inelegibilidade - Vínculo em União Estável","Nome do Candidato - Homonímia","Nome do Candidato - Variação Nominal","Prestação de Contas - De Candidato","Impugnação ao Registro de Candidatura","Registro de Candidatura - Preenchimento de Vaga Remanescente","Registro de Candidatura - RRC - Candidato","Registro de Candidatura - Substituição de Candidato - Por Cancelamento de Registro","Registro de Candidatura - Substituição de Candidato - Por Cassação de Registro","Registro de Candidatura - Substituição de Candidato - Por Expulsão Partidária","Registro de Candidatura - Substituição de Candidato - Por Falecimento","Registro de Candidatura - Substituição de Candidato - Por Indeferimento de Registro","Registro de Candidatura - Substituição de Candidato - Por Ineligibilidade","Registro de Candidatura - Substituição de Candidato - Por Renúncia","Cargo - Deputado Distrital","Cargo - Deputado Estadual","Cargo - Deputado Federal","Cargo - Governador","Cargo - Prefeito","Cargo - Presidente da República","Cargo - Primeiro Suplente de Senador","Cargo - Segundo Suplente de Senador","Cargo - Senador","Cargo - Vereador","Cargo - Vice-Governador","Cargo - Vice-Prefeito","Cargo - Vice-Presidente da República","Eleições - 1° Turno","Eleições - 2° Turno","Eleições - Anulação de Eleições","Eleições - Eleição Majoritária","Eleições - Eleição Proporcional","Eleições - Eleição Suplementar","Pesquisa Eleitoral - Divulgação de Pesquisa Eleitoral Fraudulenta","Pesquisa Eleitoral - Divulgação de Pesquisa Eleitoral Sem Prévio Registro","Pesquisa Eleitoral - Registro de Pesquisa Eleitoral","Propaganda Política - Propaganda Eleitoral - Adesivo","Propaganda Política - Propaganda Eleitoral - Alto-falante/Amplificador de Som","Propaganda Política - Propaganda Eleitoral - Banner/Cartaz/Faixa","Propaganda Política - Propaganda Eleitoral - Bem Particular","Propaganda Política - Propaganda Eleitoral - Bem Particular de Uso Comum","Propaganda Política - Propaganda Eleitoral - Bem Público","Propaganda Política - Propaganda Eleitoral - Bem Tombado","Propaganda Política - Propaganda Eleitoral - Boca de Urna","Propaganda Política - Propaganda Eleitoral - Cavalete","Propaganda Política - Propaganda Eleitoral - Comício/Showmício","Propaganda Política - Propaganda Eleitoral - Conduta Vedada a Emissora de Rádio/Televisão na Programação Normal","Propaganda Política - Propaganda Eleitoral - Confecção, Utilização ou Distribuição de Brinde","Propaganda Política - Propaganda Eleitoral - Contrariedade à Lei de Postura Municipal","Propaganda Política - Propaganda Eleitoral - Debate Político","Propaganda Política - Propaganda Eleitoral - Extemporânea/Antecipada","Propaganda Política - Propaganda Eleitoral - Folhetos/Volantes/Santinhos/Impressos","Propaganda Política - Propaganda Eleitoral - Distribuição de Tempo de Propaganda","Propaganda Política - Propaganda Eleitoral - Horário Eleitoral Gratuito/Inserções de Propaganda","Propaganda Política - Propaganda Eleitoral - Horário Eleitoral Gratuito/Programa em Bloco","Propaganda Política - Propaganda Eleitoral - Rádio","Propaganda Política - Propaganda Eleitoral - Rádio Comunitária","Propaganda Política - Propaganda Eleitoral - Televisão","Propaganda Política - Propaganda Eleitoral - Invasão de Horário Destinado a Outro Cargo/Partido/Coligação","Propaganda Política - Propaganda Eleitoral - Imprensa Escrita - Jornal/Revista/Tabloide","Propaganda Política - Propaganda Eleitoral - Inobservância do Limite Legal","Propaganda Política - Propaganda Eleitoral - Internet","Propaganda Política - Propaganda Eleitoral - Omissão de Informações Obrigatórias","Propaganda Política - Propaganda Eleitoral - Outdoors","Propaganda Política - Propaganda Eleitoral - Pintura em Muro","Propaganda Política - Propaganda Eleitoral - Truncagem, Montagem, Utilização de Gravação Externa, Computação Gráfica, Desenho Animado ou Efeito Especial","Captação ou Gasto Ilícito de Recursos Financeiros de Campanha Eleitoral","Prestação de Contas - De Comitê Financeiro","Registro de Comitê Financeiro - Eleição Majoritária","Registro de Comitê Financeiro - Eleição Proporcional","Registro de Comitê Financeiro - Único","Contas - Aprovação das Contas","Contas - Aprovação das Contas com Ressalvas","Contas - Desaprovação/Rejeição das Contas","Contas - Não Apresentação das Contas","Doação de Recursos Acima do Limite Legal - Pessoa Física","Doação de Recursos Acima do Limite Legal - Pessoa Jurídica","Habeas Corpus - Liberatório","Habeas Corpus - Preventivo","Impugnação - Identidade de Eleitor Admitido a Votar","Impugnação - Nomeação de Membro da Mesa Receptora","Impugnação - Nomeação de Membro da Junta Eleitoral","Impugnação - Sistema Eletrônico de Votação","Impugnação - Voto por Cédula","Registro de Candidatura - DRAP - Partido/Coligação","Apuração/Totalização de Votos","Quociente Eleitoral/Partidário","Abuso - De Poder Econômico","Abuso - De Poder Político/Autoridade","Abuso - Uso Indevido de Meio de Comunicação Social","Captação Ilícita de Sufrágio","Corrupção ou Fraude","Propaganda Política - Propaganda Institucional","Diplomação","Direitos Políticos - Perda dos Direitos Políticos","Direitos Políticos - Restabelecimento dos Direitos Políticos","Direitos Políticos - Suspensão de Direitos Políticos","Execução - De Astreintes","Execução - De Multa Eleitoral","Execução de Julgado","Cautelar Inominada - De Busca e Apreensão","Cautelar Inominada - De Produção Antecipada de Provas","Cautelar Inominada - Incidental","Cautelar Inominada - Preparatória","Exceção - De Coisa Julgada","Exceção - De Ilegitimidade de Parte","Exceção - De Impedimento","Exceção - De Incompetência","Exceção - De Litispendência","Exceção - De Pré-Executividade","Exceção - De Suspeição","Alteração Estatutária de Partido Político em Matéria Administrativa","Coligação Partidária - Majoritária","Coligação Partidária - Majoritária e Proporcional","Coligação Partidária - Proporcional","Convenção Partidária","Filiação Partidária - Cancelamento","Filiação Partidária - Duplicidade/Pluralidade","Filiação Partidária - Nulidade","Justificação de Desfiliação Partidária","Perda de Cargo Eletivo por Desfiliação Partidária","Fusão de Partidos Políticos","Incorporação de Partidos Políticos","Dissolução de Órgão de Direção Partidária","Partido Político - Órgão de Direção Estadual","Partido Político - Órgão de Direção Municipal","Partido Político - Órgão de Direção Nacional","Partido Político - Comissão Provisória","Propaganda Política - Propaganda Intrapartidária","Propaganda Política - Propaganda Partidária","Veiculação de Propaganda Partidária - Em Bloco","Veiculação de Propaganda Partidária - Em Inserções","Plebiscito","Requerimento","Contra o Ordenamento Urbano e o Patrimônio Cultural","Crimes contra o Ordenamento Urbano e o Patrimônio Cultural","Intimação","Citação","Oitiva","Diligências","Atos executórios","Entrada de aparelho telefônico de comunicação móvel em estabelecimento prisional","Programas de Arrendamento Residencial  PAR","Tabela Price","Empréstimo consignado","Tarifas","Capitalização e Previdência Privada","Transporte Aquaviário","Dever de Informação","Práticas Abusivas","Oferta e Publicidade","Transporte Ferroviário","Transporte Rodoviário","Infrações administrativas","Fundos","Classificação indicativa","Entidades de atendimento","Conselhos tutelares","Mineração","Reserva legal","Recursos Hídricos","Poluição","Zoneamento Ecológico e Econômico","Zona Costeira","Área de Preservação Permanente","Produtos Controlados / Perigosos","Patrimônio Cultural","Controle Externo da atividade policial","Conselho do Idoso","Conselho da Comunidade","Conselho sobre Drogas","Conselho de Direitos da Criança e Adolescente","Parcelamento do Solo","Comércio Ambulante","Operações Urbanas Consorciadas","Posturas Municipais","Segurança em Edificações","Política Agrícola","Pessoa Idosa","Pessoas com deficiência","Minorias Étnicas","Comunicação Social","Moradia","Assistência Social","Alimentação","Não Discriminação","Terceirização do SUS","Financiamento do SUS","Vigilância Sanitária e Epidemiológica","Controle Social e Conselhos de Saúde","Hospitais e Outras Unidades de Saúde","Tecidos e Partes do Corpo Humano","Recebimento de bolsa de estudos","Produto Impróprio","Combustíveis e derivados","Saneamento","Responsabilidade Fiscal","Irregularidade no atendimento","Vendas casadas","Jogos / Sorteios / Promoções comerciais","Produto Impróprio","Combustíveis e derivados","Saneamento","Utilização de bens públicos","Agências/órgãos de regulação","Sistema de proteção especial a vítimas e a testemunhas","Política fundiária e da reforma agrária","Responsabilidade fiscal","Fornecimento de Medicamentos","Unidade de terapia intensiva (UTI) ou unidade de cuidados intensivos (UCI)","Crimes Previstos no Estatuto do Torcedor","Previstos no Estatuto do Torcedor","Crédito Especial","Crédito Extraordinário","Resolução","Recomendação","Resolução Conjunta","Concurso para magistrado","Prova Objetiva","Prova Subjetiva","Prova Prática-Sentença","Prova Oral","Reserva de Vagas","Concurso para servidor","Concurso para serventia extrajudicial","Juiz Leigo","Composição de Turmas Recursais","Licença","Participação Societária","Plantão Judiciário","Residência","Substituição/Convocação em Tribunal","Violação Prerrogativa Advogado","Cargos de Direção","Órgão Especial","Fraude / Quebra de ordem cronológica","Pagamento","Vacância","Remoção","Desmembramento","Cumulação","Emolumentos","Fraude","Gratuidade","Princípio da Territorialidade","Provimento Irregular","Registro Civil de Nascimento","Registro de Imóveis","Cessão","Desvio de Função","Provimento de Cargos","Entrada de aparelho telefônico de comunicação móvel em estabelecimento prisional","Inclusão do 13º salário (gratificação natalina) no PBC","Art. 26 da Lei 8.870/1994","Art. 29, § 5º, da Lei 8.213/1991","Art. 29, II, da Lei 8.213/1991","Alteração do teto máximo para o valor do benefício previdenciário do RGPS (EC 20 e 41)","Revisão do valor do benefício no primeiro reajuste após a concessão (Art. 21, § 3º, da Lei 8.880/1994)","Deficiente","Idoso","Inspeção","Revisão/Desconstituição de Ato Administrativo","Morosidade no Julgamento do Processo","Investigação Patrimonial","Apuração de Infração Disciplinar","Acidente de Trabalho - Ressarcimento ao Erário","Difusão culposa de doença ou praga","Receptação culposa","Receptação culposa","Incêndio culposo","Incêndio culposo","Explosão culposa","Explosão culposa","Uso culposo de gás tóxico ou asfixiante","Uso culposo de gás tóxico ou asfixiante","Inundação culposa","Inundação culposa","Desabamento ou desmoronamento culposo","Desabamento ou desmoronamento culposo","Difusão culposa de doença ou praga","Desastre ferroviário culposo","Desastre ferroviário culposo","Cláusulas Abusivas","Alienação Parental","Invasão de Dispositivo Informático","Violência Doméstica Contra a Mulher","Usucapião Conjugal","Exploração Sexual","Separação de Corpos","Medidas Protetivas","Extinção das Obrigaões do Falido","Aborto","Conversão da união estável em casamento","Nulidade de ato administrativo","Usucapião de bem móvel","Adjudicação de herança","Registro Civil de Pessoas Jurídicas","Internação Compulsória","Internação Voluntária","Internação Involuntária","Encaminhamento aos pais ou responsável","Orientação e acompanhamento temporário","Matrícula e frequência obrigatória em escola oficial de ensino fundamental","Inclusão em programa de auxílio à família","Requisição de tratamento médico, psicológico ou psiquiátrico, em regime hospitalar ou ambulatorial","Inclusão em programa de auzílio, orientação e tratamento a alcoólatras e toxicômanos","Acolhimento institucional","Inclusão em programa de acolhimento familiar","Colocação em família substituta","Outras medidas de proteção","Evasão Escolar","Exploração Sexual ou Prostituição","Crimes da Lei do Planejamento Familiar","Roubo qualificado","Da Lei do Planejamento Familiar","Crimes Sobre Produção, Comércio e Transporte Clandestino de Açúcar e Álcool","Sobre Produção, Comércio e Transporte Clandestino de Açúcar e Álcool","Crime Relacionado à Exportação de Bens Sensíveis e Serviços Diretamente Relacionados","Relacionado à Exportação de Bens Sensíveis e Serviços Diretamente Relacionados","Crime da Lei Sobre Permissão de Lavra Garimpeira","da Lei Sobre Permissão de Lavra Garimpeira","Crimes da Lei que Proíbe Práticas Discriminatórias para Efeitos Trabalhistas","Da Lei que Proíbe Práticas Discriminatórias para Efeitos Trabalhistas","Crimes da Lei de Proteção à Propriedade Intelectual de Programa de Computador","Da Lei de Proteção à Propriedade Intelectual de Programa de Computador","Crimes da Lei de Locação de Imóveis Urbanos","Da Lei de Locação de Imóveis Urbanos","Crimes da Lei Geral da Copa","Da Lei Geral da Copa","Crimes do Estatuto do Torcedor","Estatuto do Torcedor","Composição Civil","Internação Compulsória","Desapropriação para Regularização de Comunidade Quilombola / Dec. 4887/2003","Análise de Crédito","Registro de Candidatura %u2013 DRAP","Registro de Candidatura - DRAP Partido/Coligação","Prestação de Contas - De Candidato","Prestação de Contas - De Comitê Financeiro","Prestação de Contas - De Exercício Financeiro","Impugnação - Identidade de Eleitor Admitido a Votar","Impugnação - Inscrição Eleitoral","Impugnação - Nomeação de Membro da Junta Eleitoral","Impugnação - Nomeação de Membro da Mesa Receptora","Impugnação - Sistema Eletrônico de Votação","Impugnação - Transferência de Domicílio Eleitoral","Impugnação - Voto por Cédula","Captação ou Gasto Ilícito de Recursos Financeiros de Campanha Eleitoral","Conduta Vedada ao Agente Público","Meios Processuais","Prestação de Contas - de Partido Político","Perda ou Modificação de Guarda","Feminicídio","Homicídio Agravado pela Prática de  Extermínio de Seres Humanos","Homicídio Agravado pelo Extermínio de Seres Humanos","Revisão de Tutela Antecipada","Suspensão do Poder Familiar","Internação Provisória","Registro de Ocorrência pela PM","Registro de Ocorrência pela PM","Requerimento de Reintegração de Posse","Requerimento de Apreensão de Veículo","Cremação/Traslado","Contra a Mulher","Contra pessoas não identificadas como mulher","Contra a mulher","Contra pessoas não identificadas como mulher","Seguro-defeso ao pescado artesanal profissional","Atos de Terrorismo","Organização Terrorista","Preparação de ato terrorista","Manutenção ou financiamento do terrorismo","Fornecimento de medicamentos","Tratamento médico-hospitalar","Unidade de terapia intensiva (UTI) ou Unidade de cuidados intensivos (UCI)","Reajuste contratual","Nomeação","Dispensa","Remoção","Remoção","Levantamento","Levantamento","Remoção","Dispensa","Nomeação","Promoção, constituição, financiamento ou integração de Organização Criminosa","Revelação de identidade, fotografia ou filmagem do colaborador","Imputação falsa de prática de infração penal ou revelação inverídica sobre estrutura de organização criminal","Descumprimento de determinação de sigilo","Recusa ou omissão de dados cadastrais, registros, documentos e informações","Provocação de tumulto ou conduta inconveniente","Apadrinhamento de Criança ou Adolescente","Crimes cometidos contra autoridade ou agente descrito nos arts. 142 e 143 da Constituição Federal, integrantes do sistema prisional e da Força Nacional de Segurança Pública","Fabrico, comércio ou detenção de arma branca ou munição","Porte de arma (branca)","Vias de fato","Omissão de cautela na guarda ou condução de animal","Perturbação do trabalho ou do sossego alheios","Simulação de Qualidade de Funcionário","Uso ilegítimo de uniforme ou distintivo","Jogo de azar","Loteria não autorizada","Jogo do bicho","Servir bebidas alcóolicas a vulneráveis","Perturbação da tranquilidade","Omissão de comunicação de crime","Recusa de dados sobre a própria identificação ou qualificação","Feminicídio","Matéria Administrativa","Proposta de Alteração de Resolução","Autorização de Divulgação de Publicidade Institucional","Cancelamento de Registro de Candidatura","Debate Eleitoral","Enquete","Execução - Cumprimento de Sentença","Fundo Partidário","Registro de Partido Político","Aposentadoria compulsória (art. 42, V, Lei Complementar 35/1979)","Remoção Compulsória","Aposentadoria Compulsória","Remoção Compulsória","Fusão de Partidos Políticos","Perda por crime contra titular do mesmo poder ou descendente (L. 13.715/2018)","Importunação Sexual","Divulgação de cena de estupro, sexo ou pornografia","Inelegibilidade - Renúncia a cargo político na pendência de representação ou petição que possa levar a outra causa de inelegibilidade","Inelegibilidade - Exclusão do exercício de profissão","Inelegibilidade - Aposentadoria Compulsória ou PAD (magistrados e membros do MP)","Inelegibilidade - Demissão do serviço público","Importunação Sexual","Divulgação de cena de estupro, sexo ou pornografia","Híbrida (Art. 48/106)","Nulidade - Ausência de publicidade de decisão","Nulidade - Ausência de Fundamentação de Decisão","Nulidade - Impedimento","Nulidade - Suspeição","Nulidade - Ausência de Nome das Partes","Nulidade - Ausência de Identificação de Advogado ou de Sociedade de Advogados","Nulidade - Ausência de Intimação do Ministério Público","Nulidade - Citação Sem Observância das Prescrições Legais","Nulidade - Título Extrajudicial Não Correspondente a Obrigação Certa, Líquida e Exigível","Nulidade - Intimação Sem Observância das Prescrições Legais","Nulidade - Título Extrajudicial Não Correspondente a Obrigação Certa, Líquida e Exigível","Nulidade - Ausência de Citação do Executado","Nulidade - Execução Instaurada Antes de Condição ou Termo","Ausência de Legitimidade para a Causa","Poderes, Deveres e Responsabilidades do Juiz","Tutela de Urgência","Tutela de Evidência","Revisão de Tutela Antecipada Antecedente","Provas em geral","Tempestividade","Proposta de Nova Resolução","Correição ordinária","Correição extraordinária","Providência","Brumadinho","Boate Kiss","Mariana","Unaí"]
                assuntos = set([x.lower() for x in assuntos]) # lower case of all assuntos

                # compare all assuntos in the list in all assuntos which will problably stay find and in the next local which he's probably stay find
                verified_assuntos = [np.nan] * len(assuntos_all)

                for j, possible_ass in enumerate(assuntos_all):
                    if(possible_ass.lower() in assuntos or partes_all[j].lower in assuntos):
                        verified_assuntos[j] = possible_ass

                lista_data = data_more_then_4.to_list()
                
                for assunto in assuntos:
                    has_hifen = re.search(r' - ', assunto)
                    if(has_hifen):
                        for i,conteudo in enumerate(conteudo_all):
                            string = r'{}'.format(assunto)
                            
                            has_assunto = re.search(string, lista_data[i].lower() ,re.IGNORECASE)
                            if (has_assunto):
                                verified_assuntos[i] = assunto
                                
                print('Verificado com sucesso') 
                
                # atribuindo os novos valores nos dados
                copy = df.copy()
                aux_assuntos = df.loc[ind_range,('Assunto')]
                assuntos_not_null = aux_assuntos.loc[~pd.Series(verified_assuntos).isnull().to_numpy()]
                df.loc[assuntos_not_null.index, ('Assunto')] = assuntos_not_null
                print('Antes os Assuntos possuíam:', copy['Assunto'].isnull().sum(),'valores nulos, e agora possuem: ', df['Assunto'].isnull().sum())
                tempo_assuntos = time.time() - tempo_assuntos
                
                # ---------------------------------------------------
                # FIM

                # ---------------------------------------------------
                # Filtrando Nomes
                print('Filtrando Nomes: ...') 

                tempo_Filtrando_Nomes = time.time()
                Autores, Reus = Search_Reus_Autores(conteudo_all, partes_all)
                tempo_Filtrando_Nomes = time.time() - tempo_Filtrando_Nomes
                
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Retirando ruídos do conteúno nos Reús
                print('Retirando ruídos dos Réus ...')
                
                filtrados = []
                for i in Reus:
                    string = ""
                    if (i == []):
                        pass
                    else:
                        if(type(i[0]) == type(string)):
                            for char in i[0]:
                                
                                if (char != '[' and char != ']' and char != ',' and char != "'" and char != "'" and char != '-'):

                                    if(string != ""):
                                        if(string[-1] == ' ' and char  == ' '):
                                            string = string[:-1] + ','
                                        else:
                                            string = string + char
                                    else:
                                        string = string + char
                    filtrados.append(string)

                filtrados_reus = []
                for i in filtrados:
                    spl = i.split(',')
                    filtrados_reus.append(spl)
                print('Retirado com sucesso!')
                
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Retirando ruídos do conteúno nos Autores
                print('Retirando ruídos dos Autores ...')
                
                filtrados_autores = []
                for i in Autores:

                    string = ""
                    if (i == []):
                        pass
                    else:
                        if(type(i[0]) == type(string)):
                            for char in i[0]:
                                if (char != '[' and char != ']' and char != ',' and char != "'" and char != "'" and char != '-' and char != '('and char != ')'):
                                        string = string + char
                                        
                    filtrados_autores.append(string)
                print('Retirado com sucesso!')
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Definindo o Gênero dos Réus
                print('Definindo o Gênero dos Réus ... ')
                
                proximos_nomes = []
                posicoes = []
                primeiros_nomes = []
                
                for i,reus in enumerate(Reus):
                    primeiro_nome = ""
                    achou_espaço = 0
                    achou_virgula = 0
                    proximo_nome = ""
                    
                    if (reus != []):
                        for j,char in enumerate(reus[0]):
                            if(char != ' ' and achou_espaço == 0):
                                primeiro_nome = primeiro_nome + char
                            else:
                                if(char == ' ' and j > 0):
                                    achou_espaço = achou_espaço + 1

                            if(char == ',' and achou_virgula > 0):
                                proximos_nomes.append(proximo_nome)
                                posicoes.append(i)
                                proximo_nome = ""
                                achou_virgula = achou_virgula + 1
                                achou_espaço = 1000

                            if(char == ',' and achou_virgula == 0):
                                proximo_nome = ""
                                achou_virgula = achou_virgula + 1
                                achou_espaço = 1000

                            if (char == ' ' and achou_espaço >= 1000):
                                achou_espaço = achou_espaço + 1

                            if(char != ' ' and char != ',' and achou_virgula > 0 and achou_espaço == 1000):
                                proximo_nome = proximo_nome + char

                    proximos_nomes.append(proximo_nome)
                    posicoes.append(i)
                    primeiros_nomes.append(primeiro_nome)
                    proximo_nome = ""
                    primeiro_nome = ""
                
                reus_classificados = [np.nan]*len(primeiros_nomes)

                for i in range(len(primeiros_nomes)):
                    list_index = []
                    string = ""
                    
                    for j in range(i, len(posicoes)):
                        if(posicoes[j] == i):
                            list_index.append(j)
                        if(posicoes[j] > i):
                            break

                    if(classify_download(primeiros_nomes[i]) == 'F'):
                        if (string == ""):
                            string = string + "'Nome': '" + filtrados_reus[i][0]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
                            reus_classificados[i] = string

                    if(classify_download(primeiros_nomes[i]) == 'M'):
                        if (string == ""):
                            string = string + "'Nome': '" + filtrados_reus[i][0]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
                            reus_classificados[i] = string
                    for j,index in enumerate(list_index):
                        
                        if(classify_download(proximos_nomes[index]) == 'F'):
                            try:
                                if (string == ""):
                                    string = string + "'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
                                    reus_classificados[i] = string
                                else:
                                    string = string + ",'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
                                    reus_classificados[i] = string
                            except:
                                print('Houve uma tentativa falha de indexação na lista filtrados_reus')
                            finally:
                                pass
                            
                        if(classify_download(proximos_nomes[index]) == 'M'):
                            try:
                                if (string == ""):
                                    string = string + "'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
                                    reus_classificados[i] = string
                                else:
                                    string = string + ",'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
                                    reus_classificados[i] = string
                            except:
                                print('Houve uma tentativa falha de indexação na lista filtrados_reus')
                            finally:
                                pass
                            
                print('Definido com sucesso!')
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Definindo o Gênero dos Réus
                print('Definindo o Gênero dos Autores ... ')
                
                primeiros_nomes_autor = []

                for i,autor in enumerate(filtrados_autores):
                    nomes = re.findall('[\wÀ-ú]+\.?', autor, re.IGNORECASE)
                    if (nomes != []):
                        
                        primeiros_nomes_autor.append(nomes[0])
                    else:
                        primeiros_nomes_autor.append("")
                        
                    
                autores_classificados = [np.nan]*len(primeiros_nomes_autor)
                for i in range(len(primeiros_nomes_autor)):
                    list_index = []
                    string = ""

                    if(classify_download(primeiros_nomes_autor[i]) == 'F'):
                        string = string + "'Nome': '" + filtrados_autores[i]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
                        autores_classificados[i] = string

                    if(classify_download(primeiros_nomes_autor[i]) == 'M'):
                        string = string + "'Nome': '" + filtrados_autores[i]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
                        autores_classificados[i] = string
                print('Definido com sucesso!')
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Definindo se os Réus são pessoas jurídicas ou não
                cidades_ibge_2019 = pd.read_excel('/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/DTB_2019/RELATORIO_DTB_BRASIL_MUNICIPIO.xls') 
                cidades_ibge_2019 = cidades_ibge_2019.loc[cidades_ibge_2019['Nome_UF'] == 'São Paulo', ('Nome_Município')].to_list() # todas as cidades do estado de sao paulo dadas pelo ibge de 2010
                cidades_ibge_2019 = set([x.lower() for x in cidades_ibge_2019]) # passando os nome das cidades para minúsculo

                print('Definindo se os Réus são pessoas jurídicas, governo ou segredo de justiça ... ')
                for i,reus in enumerate(filtrados_reus):
    
                    if(type(reus_classificados[i]) == type(np.nan)):
                        # ve se o autor é uma pessoa juridica
                        string = ""

                        for j,reu in enumerate(filtrados_reus[i]):

                            has_juridic = re.search(r'(?i)(( S( |\/|\.).*A)|(ltda)|( SA)|((.|^)Companhia( |\n))|(( |^)cia( |\.))|( [I]( |i+|\n))|(condom(i|í)nio)|(ind(u|ú)stria)|(col(e|é)gio)|(escola)|(residencial)|(financeira))',reu)
                            if(has_juridic):
                                if(string == ""):
                                    string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Jurídica'"
                                    reus_classificados[i] = string
                                else:
                                    string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Jurídica'"
                                    reus_classificados[i] = string
                            else: # senão
                                # ve se o autor esta em segredo de justiça
                                has_secret = re.search(r'([A-Z]\.( )*){2,}',reu)
                                if(has_secret):
                                    if(string == ""):
                                        string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Segredo de Justiça'"
                                        reus_classificados[i] = string
                                    else:
                                        string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Segredo de Justiça'"
                                        reus_classificados[i] = string
                                else:
                                    # ve se o auto é um governo pela regex
                                    has_gov = re.search(r'(?i)(justi(ç|a)a|p(u|ú)blica|estado|prefeitura|municipal|munic(í|i)pio|Secretaria|Instituto|Nacional|vara|judicial)',reu)
                                    if(has_gov):
                                        if(string == ""):
                                            string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                                            reus_classificados[i] = string
                                        else:
                                            string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                                            reus_classificados[i] = string
                                    else: 
                                        # ve se o autor tem um nome de município
                                        if(len(reu) < 200):

                                            for municipio in cidades_ibge_2019:

                                                has_city = reu.lower().find(municipio)

                                                if(has_city > -1):
                                                    if(string == ""):
                                                        string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                                                        reus_classificados[i] = string
                                                        break
                                                    else:
                                                        string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                                                        reus_classificados[i] = string
                                                        break
                teste_reus = df.loc[ind_range]
                teste_reus.loc[:,('Reu')] = reus_classificados
                df.loc[ind_range, ('Reu')] = teste_reus.loc[:,('Reu')]
                print('Antes os Réus possuíam:', copy['Reu'].isnull().sum(),'valores nulos, e agora possuem: ', df['Reu'].isnull().sum())
                print('Definido com sucesso!')
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Definindo se os Autores são pessoas jurídicas ou não
                print('Definindo se os Autores são pessoas jurídicas, governo ou segredo de justiça ... ')
                
                for i,autor in enumerate(filtrados_autores):

#                     if(type(autores_classificados[i]) == type(np.nan)):
                    # ve se o autor é uma pessoa juridica
                    has_juridic = re.search(r'(?i)(( S( |\/|\.).*A)|(ltda)|( SA)|((.|^)Companhia( |\n))|(( |^)cia( |\.))|( [I]( |i+|\n))|(condom(i|í)nio)|(ind(u|ú)stria)|(col(e|é)gio)|(escola)|(residencial)|(financeira))',autor)
                    if(has_juridic):
                        string = ""
                        string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Jurídica'"
                        autores_classificados[i] = string
                    else: # senão
                        # ve se o autor esta em segredo de justiça
                        has_secret = re.search(r'([A-Z]\.( )*){2,}',autor)
                        if(has_secret):
                            string = ""
                            string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Segredo de Justiça'"
                            autores_classificados[i] = string
                        else:
                            # ve se o auto é um governo pela regex
                            has_gov = re.search(r'(?i)(justi(ç|a)a|p(u|ú)blica|estado|prefeitura|municipal|munic(í|i)pio|Secretaria|Instituto|Nacional|vara|judicial)',autor)
                            if(has_gov):
                                string = ""
                                string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Governo'"
                                autores_classificados[i] = string
                            else: 
                                # ve se o autor tem um nome de município
                                if(len(autor) < 200):

                                    for j,municipio in enumerate(cidades_ibge_2019):

                                        has_city = autor.lower().find(municipio)

                                        if(has_city > -1):
                                            string = ""
                                            string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Governo'"
                                            autores_classificados[i] = string
                                            break
                                        else:
                                            has_city_in_assuntos = assuntos_all[i].find(municipio)

                                            if(has_city_in_assuntos > -1):
                                                string = ""
                                                string = string + "'Nome': '" + assuntos_all[i] + "', 'Pessoa': " + "'Governo'"
                                                autores_classificados[i] = string
                                                break
                print('Definido com sucesso!')
                
                teste_autores = df.loc[ind_range]
                teste_autores.loc[:,('Autor')] = autores_classificados
                df.loc[ind_range, ('Autor')] = teste_autores.loc[:,('Autor')]
                print('Antes os Autores possuíam:', copy['Autor'].isnull().sum(),'valores nulos, e agora possuem: ', df['Autor'].isnull().sum())
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Verificando as Classes
                print('Verificando as Classes ... ')
                tempo_Classes = time.time()
                classes = ["Procedimento Ordinário","PROCESSO CÍVEL E DO TRABALHO","SUPERIOR TRIBUNAL DE JUSTIÇA","Procedimento Comum Cível","Procedimento Sumário","Procedimentos Especiais","Procedimentos Especiais de Jurisdição Contenciosa","Anulação e Substituição de Títulos ao Portador","Apreensão e Depósito de Coisa Vendida com Reserva de Domínio","Arrolamento Comum","Arrolamento Sumário","Consignação em Pagamento","Demarcação / Divisão","Depósito","Embargos de Terceiro Cível","Habilitação","Inventário","Monitória","Nunciação de Obra Nova","Prestação de Contas - Oferecidas","Ação de Exigir Contas","Restauração de Autos","Ação Rescisória","Sobrepartilha","Usucapião","Procedimentos Especiais de Jurisdição Voluntária","Abertura, Registro e Cumprimento de Testamento","Alienação Judicial de Bens","Arrecadação das Coisas Vagas","Confirmação de Testamento","Declaração de Ausência","Especialização de Hipoteca Legal","Herança Jacente","Interdição","Organização e Fiscalização de Fundação","Separação Consensual","Tutela e Curatela -  Nomeação","Procedimentos Regidos por Outros Códigos, Leis Esparsas e Regimentos","Ação Civil Coletiva","Ação Civil de Improbidade Administrativa","Ação Civil Pública Cível","Ação Popular","Alimentos - Lei Especial Nº 5.478/68","Alteração do Regime de Bens","Alvará Judicial - Lei 6858/80","Apreensão de Embarcações","Arribadas Forçadas","Regulação de Avaria Grossa","Avarias","Busca e Apreensão em Alienação Fiduciária","Cancelamento de Naturalização","Cautelar Fiscal","Cobrança de Cédula de Crédito Industrial","Compromisso Arbitral","Consignatória de Aluguéis","Conversão de Separação Judicial em Divórcio","Correição Parcial ou Reclamação Correicional","Depósito da Lei 8. 866/94","Desapropriação","Desapropriação Imóvel Rural por Interesse Social","Despejo","Despejo por Falta de Pagamento","Despejo por Falta de Pagamento Cumulado Com Cobrança","Direta de Inconstitucionalidade","Discriminatória","Dissolução e Liquidação de Sociedade","Divórcio Consensual","Divórcio Litigioso","Dúvida","Expropriação da Lei 8.257/91","Falência de Empresários, Sociedades Empresáriais, Microempresas e Empresas de Pequeno Porte","Habeas Data","Habilitação de Crédito","Homologação de Transação Extrajudicial","Imissão na Posse","Impugnação de Crédito","Inquérito Extrajudicial","Mandado de Injunção","Mandado de Segurança Coletivo","Mandado de Segurança Cível","Naturalização","Opção de Nacionalidade","Averiguação de Paternidade","Pedido de Resposta ou Retificação da Lei de Imprensa","Protesto Formado a Bordo","Recuperação Extrajudicial","Recuperação Judicial","Registro Torrens","Relatório Falimentar","Remição do Imóvel Hipotecado","Renovatória de Locação","Restituição de Coisa ou Dinheiro na Falência do Devedor Empresário","Revisional de Aluguel","Separação Litigiosa","Sonegados","Suprimento de Idade e/ou Consentimento","Suspensão de Liminar ou Antecipação de Tutela","Suspensão de Execução de Sentença","Procedimento de Liquidação","Liquidação por Arbitramento","Liquidação de Sentença pelo Procedimento Comum","Liquidação Provisória por Arbitramento","Liquidação Provisória de Sentença pelo Procedimento Comum","Procedimento de Cumprimento de Sentença/Decisão","Cumprimento de sentença","Cumprimento Provisório de Sentença","Processo de Execução","Execução de Título Extrajudicial","Insolvência Civil","Insolvência Requerida pelo Credor","Insolvência Requerida pelo Devedor ou pelo Espólio","Embargos","Embargos à Adjudicação","Embargos à Arrematação","Embargos à Execução","Embargos de Retenção por Benfeitorias","Processo Cautelar","Alimentos - Provisionais","Apreensão de  Títulos","Arresto ","Arrolamento de Bens","Atentado","Busca e Apreensão","Caução","Cautelar Inominada","Exibição","Homologação do Penhor Legal","Justificação","Protesto","Posse em Nome do Nascituro","Produção Antecipada da Prova","Regulamentação de Visitas","Separação de Corpos","Seqüestro","Recursos","Apelação Cível","Remessa Necessária Cível","Agravos","Agravo de Instrumento","Agravo de Instrumento em Recurso Especial","Agravo de Instrumento em Recurso Extraordinário","Agravo Regimental Cível","Embargos","Embargos Infringentes","Embargos Infringentes na Execução Fiscal","Recurso Ordinário Cível","Recurso Extraordinário","Recurso Especial","Outros Procedimentos","Incidentes","Incidente De Arguição de Inconstitucionalidade Cível","Assistência Judiciária","Conflito de competência cível","Exceções","Exibição de Documento ou Coisa Cível","Impugnação ao Cumprimento de Sentença","Impugnação ao Pedido de Assistência Litisconsorcial ou Simples","Impugnação ao Valor da Causa Cível","Incidente de Falsidade","Incidente de Uniformização de Jurisprudência","Remoção de Inventariante","Oposição","Atos e expedientes","Avocatória","Habilitação para Casamento","Instrução de Rescisória","Petição Cível","Comunicação","Reclamação","Registro de Casamento Nuncupativo","Representação por Excesso de Prazo","Cartas","Carta de Ordem Cível","Carta Precatória Cível","Carta Rogatória Cível","PROCESSO CRIMINAL","Medidas Preparatórias","Notícia-Crime","Representação Criminal","Representação Criminal/Notícia de Crime","Pedido de Arquivamento em Representação Criminal","Interpelações","Notificação para Explicações","Notificação para Explicações (Lei de Imprensa)","Procedimentos Investigatórios","Termo Circunstanciado","Inquérito Policial","Auto de Prisão em Flagrante","Procedimento Comum","Ação Penal de Competência do Júri","Ação Penal - Procedimento Ordinário","Processo Especial","Processo Especial do Código de Processo Penal","Crimes de Responsabilidade dos Funcionários Públicos","Crimes de Calúnia, Injúria e Difamação de Competência do Juiz Singular","Crimes Contra a Propriedade Imaterial","Processo Sumário (Detenção)","Restauração de Autos","Processo Especial de Leis Esparsas","Crimes Ambientais","Crimes Contra a Propriedade Industrial","Crimes Contra a Propriedade Intelectual","Crimes de Imprensa","Procedimento do Juizado Especial Criminal - Sumariíssimo","Procedimento Especial da Lei Antitóxicos","Procedimento Especial dos Crimes de Abuso de Autoridade","Medidas Garantidoras","Liberdade","Liberdade Provisória com ou sem fiança","Relaxamento de Prisão","Habeas Corpus Criminal","Medidas Cautelares","Pedido de Busca e Apreensão Criminal","Pedido de Quebra de Sigilo de Dados e/ou Telefônico","Medidas Investigatórias Sobre Organizações Criminosas","Pedido de Prisão","Pedido de Prisão Preventiva","Pedido de Prisão Temporária","Pedido de Prisão/ Liberdade Vigiada para Fins de Expulsão","Questões e Processos Incidentes","Exceções","Exceção de Suspeição","Exceção de Incompetência de Juízo","Exceção de Litispendência","Exceção de Ilegitimidade de Parte","Exceção de Coisa Julgada","Exceção de Impedimento","Exceção da Verdade","Conflito de Jurisdição","Restituição de Coisas Apreendidas","Embargos de Terceiro","Medidas Assecuratórias","Seqüestro","Arresto / Hipoteca Legal","Incidentes","Incidente de Falsidade","Insanidade Mental do Acusado","Cartas","Carta de Ordem Criminal","Carta Precatória Criminal","Carta Rogatória Criminal","Execução Criminal","Execução da Pena","Incidentes","Conversão de Pena","Excesso ou Desvio","Anistia","Indulto","Comutação de Pena","Recursos","Agravo de Execução Penal","Apelação em Mandado de Segurança","Apelação Criminal","Carta Testemunhável","Correição Parcial Criminal","Embargos de Declaração Criminal","Embargos Infringentes e de Nulidade","Protesto por Novo Júri","Recurso de Medida Cautelar Criminal","Recurso de Sentença Criminal","Recurso em Sentido Estrito","Remessa Necessária Criminal","Revisão Criminal","Desaforamento de Julgamento","Incidente de Uniformização de Jurisprudência","Procedimento do Juizado Especial Cível","Pedido de Uniformização de Interpretação de Lei Cível","Recurso Inominado Cível","JUIZADOS DA INFÂNCIA E DA JUVENTUDE","Ação Anulatória de Cláusulas Convencionais","Ação de Cumprimento","Ação Trabalhista - Rito Ordinário","Inquérito para Apuração de Falta Grave","Dissídio Coletivo","Dissídio Coletivo de Greve","Execução de Título Extrajudicial","Execução de Termo de Ajuste de Conduta","Execução de Termo de Conciliação de CCP","Execução de Certidão de Crédito Judicial","Execução Provisória em Autos Suplementares","Agravo","Agravo de Instrumento em Agravo de Petição","Agravo de Instrumento em Recurso de Revista","Agravo de Instrumento em Recurso Ordinário","Agravo de Petição","Agravo Regimental Trabalhista","Embargos","Embargos Infringentes","Recurso de Revista","Recurso Ordinário Trabalhista","Agravo em Recurso de Habeas Corpus","Embargos de Divergência em Recurso de Mandado de Segurança","Embargos de Divergência em Agravo Regimental","Embargos à Execução em Mandado de Segurança","Embargos à Execução em Ação Recisoria","Embargos à Execução em Medida Cautelar","Execução em Ação Rescisória","Execução em Sentença Estrangeira","Execução em Sentença Estrangeira Contestada","Execução em Mandado de Segurança","Execução em Medida Cautelar","Incidente de Deslocamento de Competência ","Sentença Estrangeira","Sentença Estrangeira Contestada","Sindicância","Mandado de Segurança","Reclamação","Recurso Ordinário","Recurso Especial","Ação Penal","Interpelação Judicial","Apelação Cível","Suspensão de Liminar e de Sentença","Embargos Infringentes em Apelação Cível","Representação","Requisição de Pequeno Valor","Inquérito","Agravo de Instrumento","Agravo em Recurso Extraordinário","Agravo em Recurso de Mandado de Segurança","Carta Rogatória","Habeas Data","Conflito de Atribuição","Conflito de Competência","Petição","Medida Cautelar","Exceção da Verdade","Remessa Necessária em Habeas Corpus","Embargos Infringentes em Ação Rescisória","Procedimentos Trabalhistas","Processo de Execução Trabalhista","Incidentes Trabalhistas","Recursos Trabalhistas","Pedido de Revisão do Valor da Causa","Processo de Conhecimento","Procedimento de Conhecimento","Execução de Título Judicial","Execução de Alimentos","Execução Contra a Fazenda Pública","Execução Fiscal","Execução Hipotecária do Sistema Financeiro da Habitação","Embargos à Execução Fiscal","Tutela e Curatela - Remoção e Dispensa","Justificação de Dinheiro a Risco","Ação Trabalhista - Rito Sumaríssimo","Ação Trabalhista - Rito Sumário (Alçada)","Embargos de Divergência em Recurso Especial","Conflito de Competência","Incidente de Arguição de Inconstitucionalidade Criminal","PROCEDIMENTOS ADMINISTRATIVOS","Pedido de Providências","Reclamação","Agravo Interno Cível","Agravos","Exceção de Impedimento","Exceção de Suspeição","Exceção de Incompetência","Efeito Suspensivo","Processo Administrativo Disciplinar em face de Servidor","Processo Administrativo Disciplinar em face de Magistrado","Precatório","Requisição de Pequeno Valor","Recurso de Multa","Medidas Protetivas de urgência (Lei Maria da Penha) Criminal","Habeas Corpus Cível","Recurso de Medida Cautelar Cível","Superveniência de doença mental","Unificação de penas","Conflito de atribuição","Transferência entre estabelecimentos penais","Outras medidas provisionais","Reabilitação","Outros procedimentos de jurisdição voluntária","Alvará Judicial","Declaratória de Constitucionalidade","Intervenção em Município","Processo Administrativo","Recurso Administrativo","Reclamação Disciplinar","Correição Extraordinária","Inspeção","Recurso em Processo Administrativo Disciplinar em face de Servidor","Correição Ordinária","Sindicância","SUPREMO TRIBUNAL FEDERAL","Ação Cautelar","Ação Cível Originária","Ação Declaratória de Constitucionalidade","Ação Direta de Inconstitucionalidade","Ação Originária","Ação Originária Especial","Ação Penal","Ação Rescisória","Agravo Interno","Agravo de Instrumento","Agravo Regimental","Arguição de Descumprimento de Preceito Fundamental","Arguição de Suspeição","Carta Rogatória","Comunicação","Conflito de Competência","Embargos de Declaração","Embargos de Divergência","Embargos Infringentes","Extradição","Habeas Corpus","Habeas Data","Inquerito","Intervenção Federal","Mandado de Injunção","Mandado de Segurança","Medida Cautelar","Petição","Prisão Preventiva para Extradição","Processo Administrativo","Questão de Ordem","Reclamação","Recurso Criminal","Recurso em Habeas Corpus","Recurso em Habeas Data","Recurso em Mandado de Injunção","Recurso em Mandado de Segurança","Recurso Extraordinário","Revisão Criminal","Sentença Estrangeira","Sentença Estrangeira Contestada","Suspensão de Liminar","Suspensão de Segurança","Suspensão de Tutela Antecipada","Siglas dos Processos Extintos","Ação Ordinária Regressiva","Ação Regressiva","Apelação Cível","Apelação Criminal","Arguição de Relevância","Carta Testemunhável","Conflito de Atribuições","Conflito de Jurisdição","Denúncia","Embargos Remetidos","Exceção da Verdade","Exceção de Suspeição","Inquérito Administrativo","Inquérito Policial","Inquérito Policial Especial","Interpelação","Liquidação de Sentença","Notificação","Pedido de Avocação","Processo Crime","Processo Judicial","Queixa Crime","Recurso de Apreensão de Livro","Recurso de Liquidação de Sentença","Recurso de Revista","Representação","Retificação de Nome Estrangeiro","Sentença Arbitral","Suspensão de Direitos","Seção Cível","Processo de Conhecimento","Ação de Alimentos","Revisão Judicial de Decisão do Conselho Tutelar","Apuração de Irregularidades em Entidades de Atendimento","Apuração de Infração Administrativa às Normas de Proteção à Criança ou Adolescente","Tutela Infância e Juventude","Tutela c/c Destituição do Poder Familiar","Adoção","Adoção c/c Destituição do Poder Familiar","Suprimento de Capacidade ou de Consentimento para Casar","Emancipação","Regularização de Registro Civil","Guarda","Providência ","Prestação de Contas Infância e Juventude","Perda ou Suspensão do Poder Familiar","Processo de Execução","Execução de Alimentos Infância e Juventude","Execução de Medida de Proteção à Criança e Adolescente","Execução de Multa","Processos Cautelares","Busca e Apreensão Infância e Juventude","Cautelar Inominada Infância e Juventude","Cartas","Carta de Ordem Infância e Juventude","Carta Precatória Infância e Juventude","Seção Infracional","Procedimentos Investigatórios","Auto de Apreensão em Flagrante","Relatório de Investigações","Boletim de Ocorrência Circunstanciada","Processo de Apuração de Ato Infracional","Execução de Medidas Sócio-Educativas","Internação com Atividades Externas","Internação sem Atividades Externas","Semiliberdade","Liberdade Assistida","Prestação de Serviços a Comunidade","Obrigação de Reparar o Dano","Advertência ","Cartas","Carta de Ordem Infracional","Carta Precatória Infracional","Ação Rescisória","Ação de Improbidade Administrativa","Comunicação","Exceção de Impedimento","Exceção de Suspeição","Intervenção Federal","Mandado de Injunção","Precatório","Revisão Criminal","Suspensão de Segurança","Consulta Administrativa","Retificação ou Suprimento ou Restauração de Registro Civil","Retificação de Registro de Imóvel","Remessa Necessária Trabalhista","Embargos de Declaração Cível","Ação Civil Pública Infância e Juventude","Mandado de Segurança Infância e Juventude","Nomeação de Advogado","Impugnação de Assistência Judiciária","Autorização judicial","Embargos de Terceiro Infância e Juventude","Remoção, modificação  e dispensa de tutor ou curador","Procedimento Comum Infância e Juventude","Reintegração / Manutenção de Posse","Interdito Proibitório","Mandado de Segurança Criminal","Agravo de Instrumento em Recurso Especial","Agravo de Instrumento em Recurso Extraordinário","Execução Provisória","Embargos do Acusado","Alienação de Bens do Acusado","Avaliação para atestar dependência de drogas","Habeas Corpus","Recurso Ordinário em Mandado de Segurança","Recurso Ordinário em Habeas Corpus","Contraprotesto Judicial","Notificação","Interpelação","Petição Criminal","Apelação / Remessa Necessária","Agravo Regimental Criminal","Recurso Ordinário Criminal","Investigação contra magistrado","Procedimento Investigatório Criminal (PIC-MP) ","Habilitação para Adoção","Correição Parcial Cível","Ação Penal - Procedimento Sumário","Ação Penal - Procedimento Sumaríssimo","Incidente de Sanidade Mental","Medidas Protetivas - Estatuto do Idoso Criminal ","Incidentes","Conflito de Competência Infância e Juventude","Exceções","Exibição de Documento ou Coisa Infância e Juventude","Impugnação ao Valor da Causa Infância e Juventude","Incidente de Falsidade Infância e Juventude","Exceção de Impedimento Infância e Juventude","Exceção de Incompetência Infância e Juventude","Exceção de Suspeição Infância e Juventude","Petição Infracional","Cumprimento Provisório de Decisão","Impugnação ao Cumprimento  de Decisão","Petição Infância e Juventude Cível","Remessa Necessária / Recurso Ordinário","PROCESSO MILITAR","PROCEDIMENTOS ESPECIAIS PREVISTOS EM LEIS ESPARSAS","PROCESSO CRIMINAL","Procedimento Especial","Procedimentos Investigatórios","Recursos","Conselho de Justificação ","Representação p/ Declaração de Indignidade/Incompatibilidade ","Representação p/ Perda da Graduação ","Ação Penal Militar - Procedimento Ordinário","Embargos em Ação Penal Militar","Reclamação Militar","Recurso Inominado Militar","Inquérito Policial Militar","Correição Parcial Militar","Deserção","Insubmissão","Deserção de Oficial","Deserção de Praça","PROCEDIMENTOS PRÉ-PROCESSUAIS DE RESOLUÇÃO CONSENSUAL DE CONFLITOS","Extinção das obrigações do falido","Recurso em sentido estrito/Recurso ex officio","Execução de Medida de Segurança","Agravo de Instrumento em Recurso Ordinário para o STF","PROCESSO ELEITORAL","Processos Cíveis-Eleitorais","Ação de Impugnação de Mandato Eletivo","Ação de Investigação Judicial Eleitoral","Ação Penal Eleitoral","Procedimentos Relativos a Realização de Eleição","Apuração de Eleição","Prestação de Contas","Registro de Candidatura","Recurso contra Expedição de Diploma","Procedimentos Relativos a Partidos Políticos","Cancelamento de Registro de Partido Político","Propaganda Partidária","Registro de Órgão de Partido Político em Formação","Registro de Comitê Financeiro","Registro de Partido Político","Procedimentos Administrativos da Justiça Eleitoral","Representação","Correição","Criação de Zona Eleitoral ou Remanejamento","Instrução","Lista Tríplice","Revisão de Eleitorado","Recursos Eleitorais","Recurso Eleitoral","Recurso Especial Eleitoral","Recurso Ordinário Eleitoral","Consulta","Pedido de Desaforamento","Instrução Provisória de Deserção","Suspensão de Liminar e de Sentença","Suspensão de Segurança Cível","Assistência Judiciária","Exibição de Documento ou Coisa Criminal","Impugnação ao Valor da Causa","Impugnação de Assistência Judiciária","Pedido de Uniformização de Interpretação de Lei Criminal","Produção Antecipada de Provas Criminal","Restituição de Coisas Apreendidas Infracional","Processo de Aplicação de Medida de Segurança Por Fato Não Criminoso","Instrução Provisória de Insubmissão","Instrução Provisória de Deserção","Reclamação Pré-processual","Recurso Extraordinário com Agravo","Agravo em Recurso Especial","Recurso de Revista com Agravo","Recurso Ordinário - Rito Sumaríssimo","Acompanhamento de Cumprimento de Decisão","Ato Normativo","Nota Técnica","Parecer de Mérito sobre Anteprojeto de Lei","Procedimento de Controle Administrativo","Revisão Disciplinar","Reclamação para Garantia das Decisões"," Comissão","Arguição de Suspeição e de Impedimento","Cautelar Inominada Criminal","Embargos de Divergência em Agravo em Recurso Especial","Remição de Pena","Pedido de Busca e Apreensão Infracional","Reexame Necessário Eleitoral","Suspensão de Segurança","Suspensão de Segurança - Eleitoral","Exceção","Ação Cautelar","Pedido de Medida de Proteção","Procedimentos Cautelares","Busca e Apreensão Infracional","Internação Provisória","Pedido De Desinternação/Reavaliação/Substituição/Suspensão da Medida","Procedimento Conciliatório","Restabelecimento do Poder Familiar","Homologação em Acordo de Colaboração Premiada","Cumprimento de Sentença contra a Fazenda Pública","Execução de Título Extajudicial contra a Fazenda Pública","Incidente de Impedimento Cível","Incidente de Suspeição Cível","Carta Arbitral","Tutela Antecipada Antecedente","Tutela Cautelar Antecedente","Incidente de Resolução de Demandas Repetitivas","Dissolução Parcial de Sociedade","Incidente de Assunção de Competência","Liquidação Provisória de Sentença pelo Procedimento Comum","Renovação de Permanência em Estabelecimento Penal Federal","Suspensão em Incidente de Resolução de Demandas Repetitivas","Suspensão em Incidente de Resolução de Demandas Repetitivas","Incidente de Desconsideração de Personalidade Jurídica","Agravo em Execução de Medidas Sócio-Educativas","Auto de Prisão","Reclamação Criminal","Roteiro de Pena","Roteiro de Pena","Progressão de Regime","Regressão de Regime","Remição","Livramento Condicional","Outros Incidentes de Execução Iniciados de Ofício","Incidente de Julgamento de Recurso de Revista e de Embargos Repetitivos","Tutela Provisória","Tutela Cautelar Antecedente","Tutela Antecipada Antecedente","Pedido de Mediação Pré-Processual","Requerimento de Apreensão de Veículo","Requerimento de Reintegração de Posse","Contestação em Foro Diverso","Embargos Parciais à Ação Monitória","Execução de Título Extrajudicial","Suspensão Nacional do Incidente de Resolução de Demandas Repetitivas","Tutela Provisória Antecedente","Tutela Provisória Incidental","Prestação de Contas Eleitorais","Notificação","Interpelação","Protesto","Protesto formado a bordo","Guarda c/c destituição do poder familiar","Cumprimento de sentença - Lei Arbitral (Lei 9.307/1996)","Ação de Partilha","Tutela Cível","Curatela","Cumprimento de Sentença de Obrigação de Prestar Alimentos","Execução Extrajudicial de Alimentos","Pedido de Cooperação Jurisdicional","Suspensão em IRDR","Suspensão em IRDR","Execução de Título Judicial - CEJUSC","Pedido de Efeito Suspensivo à Apelação","Tomada de Decisão Apoiada","Tomada de Decisão Apoiada","Alteração de Regime de Bens","Divórcio Consensual","Divórcio Litigioso","Homologação da Transação Extrajudicial","Reclamação","Regulação de Avaria Grossa","Prestação de Contas - Partidos Políticos","Incidente de Impedimento Infância e Juventude","Incidente de Suspeição Infância e Juventude","Ação de Partilha","ação de partilha","Incidente de Uniformização de Jurisprudência","Revisão Criminal","Medidas Protetivas de Urgência (Lei Maria da Penha) Infracional","Medidas Protetivas - Estatuto do Idoso Infracional","Corregedoria Eleitoral","Petição Corregedoria","Inquérito Administrativo"]
                classes = [classe.lower() for classe in classes]
                
                # compare all assuntos in the list in all assuntos which will problably stay find and in the next local which he's probably stay find

                verified_classes = [np.nan] * len(classes_all)

                for j, possible_class in enumerate(classes_all):
                    
                    if(possible_class.lower() in classes or classes_all[j].lower in classes):
                        verified_classes[j] = possible_class
                
                for classe in classes:
                    has_hifen = re.search(r' - ', classe)
                    if(has_hifen):
                        for i,conteudo in enumerate(conteudo_all):
                            string = r'{}'.format(classe)
                            
                            has_classe = re.search(string, lista_data[i].lower() ,re.IGNORECASE)
                            if (has_classe):
                                verified_classes[i] = classe
                                
                teste_classes = df.loc[ind_range]
                teste_classes.loc[:,('Classe')] = verified_classes
                df.loc[ind_range, ('Classe')] = teste_classes.loc[:,('Classe')]
                tempo_Classes = time.time() - tempo_Classes
                
                print('Antes as Classes possuíam:', copy['Classe'].isnull().sum(),'valores nulos, e agora possuem: ', df['Classe'].isnull().sum())
                print('Verificado com sucesso!')
                
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Buscando Valores no conteúdo
                print('Buscando valores nos documentos ...')
                tempo_Valores = time.time()
                
                verified_valores = [np.nan] * len(conteudo_all)

                for i,conteudo in enumerate(conteudo_all):
                    string = ""
                    found_valor = re.findall(r'R\$[ |]?[0-9]*[\.|,|]?[0-9]+,[0-9]+', conteudo, re.IGNORECASE)

                    if(found_valor):

                        for f in found_valor:
                            nums = re.findall(r'[0-9]*[\.|,|]?[0-9]+,[0-9]+', f, re.IGNORECASE)
                            if(string == ""):
                                string = "'Valores': " + "'R$': '" + nums[0] + "'"
                                verified_valores[i] = string
                            else:
                                string = string + ",'R$': '" + nums[0] + "'"
                                verified_valores[i] = string

                df.loc[ind_range,'Valores'] = verified_valores 
                tempo_Valores = time.time() - tempo_Valores
                print('Encontrados: ')
                
                print('Foram encontrados:', chunksize - df['Valores'].isnull().sum(),'valores presentes nos documentos')
                
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Salvando em novo arquivo json
                print(df.isnull().sum())
                # escrevendo no documento:
                to_json_append(df,'{}.json'.format(file))
                j = df.index[-1]+1

                print('| index: {}'.format(j))
                # ---------------------------------------------------
                # FIM
                
                # ---------------------------------------------------
                # Escrevendo logs estatísticos na pasta log desse chunk
                path = 'Logs dos documentos/'
                f=open(path+file,'a+', encoding='utf-8')
                string = "# -------------------------------------------------------------------------------------------------\nEstatísticas Neste Chunk: \n \nTamanho do Chunk: {} \n\n".format(chunksize)
                
                texto = 'Arquivo: '+ file + "\n" + \
                        'Assuntos: possuíam {} valores nulos, e agora possuem: {}\n'.format(copy['Assunto'].isnull().sum(), df['Assunto'].isnull().sum()) + \
                        'Autores: possuíam {} valores nulos, e agora possuem: {}\n'.format(copy['Autor'].isnull().sum(), df['Autor'].isnull().sum()) + \
                        'Classes: possuíam {} valores nulos, e agora possuem: {}\n'.format(copy['Classe'].isnull().sum(), df['Classe'].isnull().sum()) + \
                        'Réus: possuíam {} valores nulos, e agora possuem: {}\n'.format(copy['Reu'].isnull().sum(), df['Reu'].isnull().sum()) + \
                        'Valores: Foram encontrados: {} valores presentes nos documentos \n'.format(chunksize - df['Valores'].isnull().sum())
                summario = df.isnull().sum()
                f.write(string)
                f.write(texto)
                f.write(str(summario) + '\n \n')
                f.write('Tempos: \n\
    Tempo total decorrido para separar   os " - "  separadores no conteudo         : {:.2f}, \n\
    Tempo total decorrido para separar   assuntos, partes,     classes, etc...     : {:.2f}, \n\
    Tempo total decorrido para verificar os        processos   no conteudo         : {:.2f}, \n\
    Tempo total decorrido para verificar os        assuntos    no conteudo filtrado: {:.2f}, \n\
    Tempo total decorrido para filtrar   os Réus   e  Autores  no conteudo filtrado: {:.2f}, \n\
    Tempo total decorrido para verificar as        Classes     no conteudo filtrado: {:.2f}, \n\
    Tempo total decorrido para encontrar valores   no          conteudo    filtrado: {:.2f}, \n\
    nesse chunk \n\n'.format(tempo_sepadores, tempo_Separar_tudo, tempo_numero_processo,tempo_assuntos, tempo_Filtrando_Nomes, tempo_Classes, tempo_Valores))
                f.write('Tempo total decorrido para executar esse chunk: {} segundos \n\n'.format(time.time() - time_inicio_chunk))
                
                f.close()
                # ---------------------------------------------------
                # FIM


  0%|          | 0/355 [00:00<?, ?it/s]

Executando: andradina_foro_de_andradina.json
Verificando assuntos ...
